# : : : : GESTION DE OT : : : : :
🔥version 0.10.0

    🚰 Eliminar el colaborador si es el mismo que el Responsable.

✅ PyMuPDF ver: 1.24.9

✅ LOGGING: for Debuggin'
    
- Si hay error en una tabla, logger el recuadro "box" con get_text()
- Loggear a un archivo las novedades encontradas durante la importacion.


✅ pyMongo DB (basic)

🚰 pyMongo Driver
- Falta definir el modelo de trabajo para cambiar/reemplazar/modificar una o varias ot
- Verificar si existe antes de la inserción, no sobreescribir

🟤 DASK / Multi-processing

---

### ⚠ Problemas conocidos:

* No puede haber dos Riesgos existentes iguales
    - Se reescribe su llave "Key":
    - Ej: si hay dos riescos `PSICOSOCIALES`, el segundo sobreescribe al primero.
    
    - Esto se debe a la naturaleza de la estructura del dato, `dictionary`

* En Actividades, las lineas desbordadas se las encuentra en el item de abajo.
    - Cuando se tiene más de tres lineas en una actividad, la cuarta línea "se pasa" a la siguiente actividad. Esto debe ser corregido en el PDF.




### OBJETIVO:

1. Transformar los informes diarios de actividades desaroolladas por los grupos de trabajo de la Jefatura Zonal Zamora, conocidos como 'Orden de Trabajo' del formato PDF a formato JSON y desde ahía a un DataFrame para su posterior análisis.

2. Anotar pas partes del PDF encontrado: [streamlit-pdf-viewer](https://github.com/lfoppiano/streamlit-pdf-viewer) como parte de la Calificación.

version 0.9.1
- Identificar las tres partes principales: DICTIONARIO / MATRIZ / EXCEL

version 0.8.0
- Requiere Java SDK «Tabula» ☝ ▶ [reemplazar por PyMuPDF](https://pymupdf.readthedocs.io/en/latest/)
- PyPDF2 se utiliza para verificar que se trata de un archivo PDF que sea una Órden de trabajo válido antes de procesar.  ☝ ▶ `reemplazar por MuPDF`
- Guardar DB Dictionary (JSON) en MongoDB Atlas como una primera aproximación a DataLake.
- Los Archivos procesados se guardan en formato `PICKLE`



------------------


- `[TESTING]` Información de Responsable: [NOMBRE,CARGO]
- `[TESTING]` Información de los Colaboradores con el cargo:

- Verificar que la variable fechaModa cumpla con el formato 2023-12-01 00:00:01
- Si una fecha de actividad no coincide con la fecha moda, esta deberá ser cambiada
- Desde el 01/Feb/2024 se ha añadido soporte para multiples páginas de actividades, posiblemente solo dos, esto aún no es soportado por este Script.
- Explorar la opcion de usar: [PyMuPDF](https://artifex.com/blog/table-recognition-extraction-from-pdfs-pymupdf-python) : Puede resultar más rápido pero se necesita invertir mucho más tiempo en la programación.
- Cargar nuevas OT a MongoDB  ⚠  **Crear un Driver para MongoDB**
- Tabula-Py no puede procesar el Archivo: 4_page1_full_no_activities.pdf


#1.1 - Definiciones
Definiciones globales y directorios

In [ ]:
"""
    Nombre con el que se Guardarán los archivos Generados en formato PICKLE.

    Este nombre debe variar dependiendo del proyecto.
"""

VERSION = "0.10.0"

file_prefix = '2024-09_septiembre'  # Importante cambiarlo para evitar sobreescrituras



# Directorio donde se guardaran los archivos generados. Importante el '/' al final

save_dir = '/content/drive/MyDrive/Colab_Disk/OT/'

# Combinaciones de nombres con el Prefijo y su Sufijo los cuales de guardaran en el
# Directorio de destino

path_raw    = save_dir + file_prefix + '_0_raw.pkl'
path_dict   = save_dir + file_prefix + '_a_dict.pkl'
path_matriz = save_dir + file_prefix + '_b_matriz.pkl'
path_log    = save_dir + file_prefix + '_c_log.txt'
path_excel  = save_dir + file_prefix + '_d_excel.xlsx'



```
    El archivo 'raw' y 'dict' tienen los mismo datos, lo más fieles posible
    a los archivos PDF, a partir de estos datos hacemos el procesamiento.

    Con estos archivos podemos Revisar, detectar potenciales errores.
    Esto con la finalidad de corregirlos antes de TERMINAR una OT.

    A partir de estos datos los procesamos hasta obtener una matriz 2D.
    Aplicando correcciones de errores comunes y fáciles de corregir, en el
    caso de que se haya corregido previamente.
    Esta matriz será guardada en un archivo PICKLE.

    Desde el Archivo Matriz podemos CALIFICAR las actividades y proceder a
    exportarlo a Excel para una revisión y corrección manual, luego volverlo
    a cargar ya que desde este archivo matriz una vez corregido se generarán
    los informes y reportes necesarios.

```


# 1 - EXTRACCIÓN
Funciones y definiciones para convertir de PDF a JSON (dictionary)

```
"""
    Directorio donde se encuentran las OT's para convertir.

    Esta será la carpeta en Google Drive donde se buscarán
    todos los archivos PDF, filtrados aquellos válidos como
    Orden de Trabajo y procesados.
"""
```

In [ ]:

##     *** CARPETA RAÍZ DONDE SE ENCUENTRAN LOS PDF Para convertir ***     ##

pdf_dir ='/content/drive/MyDrive/Colab_Disk/OT/2024/08 AGOSTO'



## 1.1 PyMuPDF (Fitz)



### Nombres y Librerias


In [ ]:
!pip install PyMuPDF==1.24.9
!pip install icecream==2.1.3
!pip install pymongo
!pip install "dask[complete]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 14.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 17.4 MB/s eta 0:00:00


In [ ]:
import os
from   os.path import exists
from   os.path import basename
from   pathlib import Path

from operator import index # 11 FechaInicio_Testimado

from datetime import datetime
from pytz import timezone

import dask
from   dask import delayed
import pickle
import json
import pymongo

import pandas as pd
import numpy as np

import pymupdf
from   pprint import pprint
from   pymupdf import Rect

print(pymupdf.__version__)

from icecream import ic
ic.configureOutput(prefix='\not_pdf| ', includeContext=True)


# VALUES OF LINES FOR TABLE AND TEXT EXTRACTION
#    py1  <- Vertical Values in PPi for the first paeg
#    px1  <- Horizontal Values in PPi for the first page
#    ry1  <- Vertical Values in PPi for the second page

#    PPi points per inch : 72 points per inch


py1 = 25.90
py2 = 42.00
py3 = 54.00
py4 = 65.50
py5 = 81.50
py6 = 93.00
py7 = 182.00
py8 = 195.00
py9 = 230.00
py10 = 215.27
py11 = 239.55
py12 = 274.50
py13 = 286.00
py14 = 363.00
py15 = 378.00
py16 = 545.00
py17 = 560.00
py18 = 572.00
py19 = 738.50
py20 = 793.65
py21 = 763.00
py22 = 837.67
py23 = 845.77

py24 = 263.00
py25 = 575.00
py26 = 580.00

py27 = 106.00
py28 = 116.00


px1 = 27.52
px2 = 84.00
px3 = 205.56
px4 = 228.50
px5 = 286.50
px6 = 270.30
px7 = 340.00
px8 = 346.38
px9 = 503.00
px10 = 567.76

px11 = 57.00
px12 = 385.00
px13 = 418.00

px14 = 225.00
px15 = 108.00
px16 = 210.00

px17 = 420.00
px18 = 465.00

px19 = 254.00
px20 = 338.00

ry1 = 25.90
ry2 = 45.00
ry3 = 55.50
ry4 = 68.50
ry5 = 72.84
ry6 = 73.00
ry7 = 687.00
ry8 = 715.00
ry9 = 726.31
ry10= 737.00
ry11= 748.00
ry12= 749.00
ry13= 770.00



# boxes hoja 1 : primera cara

bx_id_ot                  = Rect( px9 ,py1  , px10, py2 )
bx_numero_ot              = Rect( px9 ,py2  , px10, py3 )
bx_gerencia               = Rect( px5 ,py3  , px9 , py4 )
bx_sitio                  = Rect( px11,py4  , px12, py5 )
bx_fechaInicial_1         = Rect( px13,py4  , px10, py5 )


bx_responsable            = Rect( px2 ,py27  , px20, py28 )

bx_nombresColaboradores   = Rect( px2 ,py28  , px19, py7 )
bx_cargosColaboradores    = Rect( px19,py28  , px20, py7 )

bx_vehiculoHoja1          = Rect( px7 ,py6  , px10, py7 )
bx_tipos_trabajo          = Rect( px1 ,py8  , px10, py9 )
bx_descripcion            = Rect( px1 ,py11 , px10, py24)
bx_fechaInicio_Testimado  = Rect( px1 ,py24 , px10, py12)
bx_riesgos_epps           = Rect( px1 ,py13 , px10, py14)
bx_medidas_seg            = Rect( px1 ,py15 , px10, py16)
bx_precauciones           = Rect( px1 ,py17 , px10, py18)
bx_carencias              = Rect( px1 ,py25 , px10, py26)
bx_firmas                 = Rect( px1 ,py19 , px10, py21)


# boxes hoja 2: reverso

bx_cuadrilla              = Rect( px1 ,ry1  , px14,ry2 )
bx_vehiculo               = Rect( px1 ,ry2  , px10,ry3 )
bx_kilometraje            = Rect( px1 ,ry3  , px10,ry4 )
bx_actividades            = Rect( px1 ,ry6  , px10,ry7 )
bx_observaciones          = Rect( px15,ry7  , px10,ry8 )
bx_terminado              = Rect( px15,ry9  , px16,ry10)
bx_fechaFin               = Rect( px15,ry10 , px16,ry11)
bx_accidentes             = Rect( px17,ry12 , px18,ry13)



1.24.9


### Align Boxes

This section is used to align the values for the boxes, in order to find and extract tabular data.

In [ ]:
# Importarmos OT # 78,292 - Cesar Lituma del 25 de Diciembre del 2021

doc = pymupdf.open( '/content/drive/MyDrive/Colab_Disk/GestionOT/TestDirectories/0_ImportPdfOT/11_LM_Tres_hojas.pdf' )
hoja1 = doc[0]
hoja2 = doc[1]

# Sin colaboradores sin vehiculo
noCar = pymupdf.open('/content/drive/MyDrive/Colab_Disk/GestionOT/TestDirectories/0_ImportPdfOT/20_0 colaboradores sin vehiculo.pdf')
hj1 = noCar[0]
hj2 = noCar[1]

In [ ]:


# Draw Boxes on the pages

page = hoja1
shape = page.new_shape()

shape.draw_rect( bx_id_ot          )
shape.draw_rect( bx_numero_ot      )
shape.draw_rect( bx_gerencia       )
shape.draw_rect( bx_sitio          )
shape.draw_rect( bx_fechaInicial_1 )

shape.draw_rect( bx_responsable       )
shape.draw_rect( bx_nombresColaboradores )
shape.draw_rect( bx_cargosColaboradores )


shape.draw_rect( bx_vehiculoHoja1  )
shape.draw_rect( bx_tipos_trabajo  )
shape.draw_rect( bx_descripcion )

shape.draw_rect( bx_fechaInicio_Testimado )

shape.draw_rect( bx_riesgos_epps )
shape.draw_rect( bx_medidas_seg )
shape.draw_rect( bx_precauciones )
shape.draw_rect( bx_carencias )
shape.draw_rect( bx_firmas )


shapeActividades = hoja2.new_shape()

shapeActividades.draw_rect( bx_cuadrilla )
shapeActividades.draw_rect( bx_vehiculo  )
shapeActividades.draw_rect( bx_kilometraje )
shapeActividades.draw_rect( bx_actividades )
shapeActividades.draw_rect( bx_observaciones )
shapeActividades.draw_rect( bx_terminado )
shapeActividades.draw_rect( bx_fechaFin )
shapeActividades.draw_rect( bx_accidentes )


shape.finish(color=(1, 0, 0), fill=None)  # Set color and fill (red in this case)
shapeActividades.finish(color=(1, 0, 0), fill=None)  # Set color and fill (red in this case)
shape.commit()  # Commit the changes to the page
shapeActividades.commit()

doc.save('VERSION PyMuPDF Boxes.pdf')

### 2. Funciones get para extraer datos Extraer datos

### Hoja 1

#### (1) id_ot
Siempre debería tener numeración, el no obtener un respuesta válida puede deberse a un PDF mal formado o que no es de tipo "Orden de Trabajo" y debería abortarse las siguientes operaciones.

In [ ]:
# Primera tabla,

def getId_Ot( hoja ):
  try:
    texto = hoja.get_text( clip = bx_id_ot )
    id_ot = texto
    id_ot = id_ot.split('\n')[1].replace(',',"")
    id_ot = int(id_ot)
  except:

    ic({"archivo: ": hoja, "variable: ":texto})
    id_ot = "·"
  return id_ot


In [ ]:
# Test with hoja1

id_ot = getId_Ot( hoja1 )
id_ot

133130

#### (2) numeracion

puede resultar vacia, en cuyo caso se lee '0'

In [ ]:
# Function to getNumeracion

def getNumeracion( hoja ):
  try:
    texto = hoja.get_text( clip = bx_numero_ot )
    numeracion = texto
    numeracion = numeracion.replace('NM:', "").replace(',', "").strip()
    numeracion = int(numeracion)
  except:
    ic({"archivo: ": hoja, "variable: ":texto})
    numeracion = 0
  return numeracion

In [ ]:
# Test getNumeracion with hoja1

numeracion = getNumeracion( hoja1 )
numeracion

142

#### (3) Gerencia

In [ ]:
# Function to getGerencia

def getGerencia( hoja ):
  try:
    texto = hoja.get_text( clip = bx_gerencia ).strip()
    gerencia = texto
  except:
    ic({"archivo: ": hoja, "variable: ":texto})
    gerencia = "·"
  return gerencia

In [ ]:
# Test getGerencia with hoja1

gerencia = getGerencia( hoja1 )
gerencia

'OPERACIÓN Y MANTENIMIENTO'

#### (4) Sitio

Lugar de trabajo

In [ ]:
# Functio to getSitio using get_text()

def getSitio( hoja ):
  try:
    sitio = hoja.get_text( clip = bx_sitio ).strip()
  except:
    ic({"archivo: ": hoja, "variable: ":sitio})
    sitio = "·"
  return sitio

In [ ]:
# Test getSitio with hoja1

sitio = getSitio( hoja1 )
sitio

'El Pangui, Pachicutza, San Roque, El Padmi, Los Encuentros, Muchime, Chimbutza'

#### (5) Fecha de inicio

Fecha de inicio en la hoja 1 arriba a la derecha

In [ ]:
# Funtion to getFechaInicio usint get_text()

def getFechaInicio( hoja ):
  try:
    fechaInicio = hoja.get_text( clip = bx_fechaInicial_1 ).strip()
  except:
    ic({"archivo: ": hoja, "variable: ":fechaInicio})
    fechaInicio = "·"
  return fechaInicio

In [ ]:
# Test getFechaInicio usint hoja1

fechaInicio = getFechaInicio( hoja1 )
fechaInicio

'jueves, 04 de julio del 2024'

In [ ]:
# Funcion que transforma una fecha de String largo a toDateString compatible
# Si no es posible la transformacion, devuelve la misma cadena sin alterar

def toDateEcuador( fecha ):
    try:
        meses_a_numero = {
            'enero' : '01',
            'febrero' : '02',
            'marzo' : '03',
            'abril' : '04',
            'mayo' : '05',
            'junio' : '06',
            'julio' : '07',
            'agosto' : '08',
            'septiembre' : '09',
            'octubre' : '10',
            'noviembre' : '11',
            'diciembre' : '12'
        }

        fechaString = fecha.lower()

        fechaString = (
            fechaString
            .split(',')[1]
            .strip()
            .replace('del ', '')
            .replace('de ', '')
            )

        for mes, numero in meses_a_numero.items():
            fechaString = fechaString.replace(mes, numero)

        respuesta = fechaString.split(' ')
        respuesta = '/'.join(respuesta)
        date_object = datetime.strptime(respuesta, '%d/%m/%Y')
        ecuador = timezone("America/Guayaquil")
        local_datetime = ecuador.localize(date_object)


    except:
        ic({"archivo: ": fecha, "variable: ": fechaString})
        local_datetime = fecha

    return local_datetime

In [ ]:
test = 'sábado, 25 de Julio del 2021'
rta = toDateEcuador( test )
rta

datetime.datetime(2021, 7, 25, 0, 0, tzinfo=<DstTzInfo 'America/Guayaquil' -05-1 day, 19:00:00 STD>)

In [ ]:

# Define the date string
date_string = '25/07/2021'

# Convert the string to a datetime object
date_object = datetime.strptime(date_string, '%d/%m/%Y')
pacific = timezone("America/Guayaquil")
local_datetime = pacific.localize(date_object)

print(local_datetime)

2021-07-25 00:00:00-05:00


In [ ]:
# Function getDiaSemana: try and split by ',' return the first part except return '·'

def getDiaSemana( fechaInicio ):
  try:
    diaSemana = fechaInicio.split(',')[0]
  except:
    ic({"archivo: ": fechaInicio, "variable: ":diaSemana})
    diaSemana = "·"
  return diaSemana


In [ ]:
# Test getDiaSemana using getFechaInicio(hoja1)

diaSemana = getDiaSemana( fechaInicio )
diaSemana

'jueves'

#### (6) Responsable

Es la primera persona del listado



In [ ]:
def getResponsable( hoja ):
    try:
        texto = hoja.get_text( clip = bx_responsable)
        df_personal = texto
        df_personal = df_personal.strip().split('\n')
        responsable = [df_personal[0],df_personal[1]]
    except:
        ic({"archivo: ": hoja, "variable: ":texto})
        responsable = "·"
    return responsable


In [ ]:
# Test getResponsable

responsable = getResponsable( hoja1 )
responsable

['MORALES RIVERA LUIS ALBERTO', 'JECU']

#### (7) Colaboradores

Array con los colaboradores

```
    {
        Total: 3
        Colaboradores: [ #array de arrays
            [NOMBRE,CARGO],
            [NOMBRE,CARGO],
            [NOMBRE, CARGO]
        ]
        
    }

    # Sin colaboradores

    {
        Total: 0
        Colaboradores:[]
    }
```


In [ ]:
listado = hoja1.get_text( clip = bx_nombresColaboradores )
listado = listado.strip().split('\n')
listado

['ROBLES JAEN SERVIO VINICIO',
 'SARMIENTO HURTADO PABLO DANIEL',
 'YAURE DIAZ RAMIRO GABRIEL']

In [ ]:
listado = hoja1.get_text( clip = bx_cargosColaboradores )
listado = listado.strip().split('\n')
listado

['CHF2', 'LNI1', 'LI3E']

In [ ]:
def getColaboradores( hoja ):
    try:
        texto = hoja.get_text( clip = bx_nombresColaboradores )
        df_personal = texto
        df_personal = df_personal.strip().split('\n')
        df_personal = [ item for item in df_personal if item != '' ]

        data = hoja.get_text( clip = bx_cargosColaboradores )
        df_cargos = data
        df_cargos = df_cargos.strip().split('\n')
        df_cargos = [ item for item in df_cargos if item != '' ]

        totalColaboradores = len(df_personal)

        colaboradores = []
        for i in range(totalColaboradores):
            colaboradores.append([df_personal[i],df_cargos[i]])
    except:
        ic({"archivo: ": hoja, "Nombres: ":texto, "Cargo: ":data})
        totalColaboradores = 0
        colaboradores = []

    respuesta = {}
    respuesta['total'] = totalColaboradores
    respuesta['nombres'] = colaboradores
    return respuesta

In [ ]:
# Test getColaboradores

colaboradores = getColaboradores( hoja1 )
colaboradores

{'total': 3,
 'nombres': [['ROBLES JAEN SERVIO VINICIO', 'CHF2'],
  ['SARMIENTO HURTADO PABLO DANIEL', 'LNI1'],
  ['YAURE DIAZ RAMIRO GABRIEL', 'LI3E']]}

#### (8) Vehículo

esta función retorna tres valores:

- vehículo
- propietario
- chofer

In [ ]:
# Function getVehiculo using to_pandas()

def getVehiculo( hoja ):
  try:
    texto = hoja.get_text( clip = bx_vehiculoHoja1 ).strip().split('\n')
    df_vehiculo = texto

    if df_vehiculo[3] == 'Placa:':  # Hay un vehículo

      data_vehiculo={}
      data_vehiculo['numero'] = df_vehiculo[2]
      data_vehiculo['placa'] = df_vehiculo[4]
      data_vehiculo['marca'] = df_vehiculo[6]
      data_vehiculo['rentado'] = df_vehiculo[8]
      data_vehiculo['propietario'] = df_vehiculo[10]

      if len(df_vehiculo) == 13:
        data_vehiculo['chofer'] = df_vehiculo[12]
      else:
        data_vehiculo['chofer'] = "·"

    else:
      data_vehiculo={}
      data_vehiculo['numero'] = "·"
      data_vehiculo['placa'] = "·"
      data_vehiculo['marca'] = "·"
      data_vehiculo['rentado'] = "·"
      data_vehiculo['propietario'] = "·"
      data_vehiculo['chofer'] = "·"

  except:
    ic({"archivo: ": hoja, "variable: ":texto})
    data_vehiculo = "·"

  return data_vehiculo


In [ ]:
# Test getVehiculo

vehiculo = getVehiculo( hoja1 )
vehiculo

{'numero': '2-91',
 'placa': 'LEA-1595',
 'marca': 'ESPECIAL HINO FC9JJSA AC 5.1 2P 4X BLANCO',
 'rentado': 'NO',
 'propietario': 'EERSSA',
 'chofer': 'ROBLES JAEN SERVIO VINICIO'}

In [ ]:
hoja1_vehiculo = hoja1.get_text( clip = bx_vehiculoHoja1 ).strip().split('\n')
hoja1_vehiculo

['MOVILIZACIÓN',
 'Número/Código de vehículo:',
 '2-91',
 'Placa:',
 'LEA-1595',
 'Descripción:',
 'ESPECIAL HINO FC9JJSA AC 5.1 2P 4X BLANCO',
 'Rentado:',
 'NO',
 'Propietario:',
 'EERSSA',
 'Chofer:',
 'ROBLES JAEN SERVIO VINICIO']

In [ ]:
no_vehiculo = hj1.get_text( clip = bx_vehiculoHoja1 ).strip().split('\n')
no_vehiculo

['MOVILIZACIÓN',
 'Número/Código de vehículo:',
 'Placa:',
 'Descripción:',
 'Rentado:',
 'Propietario:',
 'Chofer:']

#### (9) Tipos de Trabajo

Array con los tipos de trabajo

In [ ]:
# Function getTiposTrabajo using to_pandas()

def getTiposTrabajo( hoja ):
  try:
    df_tipos_trabajo = hoja.find_tables( clip = bx_tipos_trabajo )
    df_tipos_trabajo = df_tipos_trabajo.tables[0].to_pandas()
    df_tipos_trabajo = df_tipos_trabajo.to_numpy().flatten()
    df_tipos_trabajo = [ item for item in df_tipos_trabajo if item != '' ]
  except:
    ic(hoja)
    df_tipos_trabajo = "·"
  return df_tipos_trabajo

In [ ]:
# Test tipos de trabajo

testTrabajos = getTiposTrabajo( hoja1 )
testTrabajos

['A - SIN VOLTAJE',
 'TRABAJO EN ALTURA - SOBRE NIVEL',
 'CON ENERGÍA/VOLTAJE - BAJO VOLTAJE',
 'TRABAJO EN ALTURA - BAJO NIVEL']

In [ ]:
tipos_trabajo = hoja1.find_tables( clip = bx_tipos_trabajo )
tipos_trabajo = tipos_trabajo.tables[0].to_pandas()
tipos_trabajo = tipos_trabajo.to_numpy().flatten()
tipos_trabajo = [ item for item in tipos_trabajo if item != '' ]
tipos_trabajo

['A - SIN VOLTAJE',
 'TRABAJO EN ALTURA - SOBRE NIVEL',
 'CON ENERGÍA/VOLTAJE - BAJO VOLTAJE',
 'TRABAJO EN ALTURA - BAJO NIVEL']

#### (10) Descripcion

Texto con la descripción del trabajo a realizar

In [ ]:
# Function getDescripcion using get_text()

def getDescripcion( hoja ):
  try:
    descripcion = hoja.get_text( clip = bx_descripcion ).strip()
  except:
    ic({"archivo: ": hoja, "variable: ":descripcion})
    descripcion = "·"
  return descripcion

In [ ]:
# Test getDescripcion

descripcion = getDescripcion( hoja1 )
descripcion

'En El Pangui, Pachicutza, San Roque, El Padmi, Los Encuentros, Muchime, Chimbutza, arreglar luminarias en función\ndel recorrido realizado por el Sr. Servio Barrazueta'

#### (11) FechaInicio_Testimado

Comprobar dos valores, que exista tiempo de Incio y que exista Tiempo de duración estimado

In [ ]:
# Function getFechaInicio2 with bx_fechaInicio_Testimado

def getFechaInicio2( hoja ):

  find_word_index = lambda word_list, target_word: word_list.index(target_word) if target_word in word_list else -1

  try:
    texto = hoja.get_text( clip = bx_fechaInicio_Testimado ).strip().split('\n')
    fechaInicio2 = texto
    index = find_word_index(fechaInicio2, 'TIEMPO ESTIMADO DE DURACIÓN (HORAS):')

    if index != -1:
      fechaInicio2 = fechaInicio2[index - 1]
    else:
      ic(fechaInicio2)
      fechaInicio2 = "·"
  except:
    ic({"archivo: ": hoja, "variable: ":texto})
    fechaInicio2 = "·"
  return fechaInicio2

In [ ]:
# Test getFechaInicio2 using hoja1

fechaInicio2 = getFechaInicio2( hoja1 )
fechaInicio2


'jueves, 04 de julio del 2024'

In [ ]:
# from operator import index
# Function getTestimado with bx_fechaInicio_Testimado

def getTiempoEstimado( hoja ):

  find_word_index = lambda word_list, target_word: word_list.index(target_word) if target_word in word_list else -1

  try:
    texto = hoja.get_text( clip = bx_fechaInicio_Testimado ).strip().split('\n')
    Testimado = texto
    index = find_word_index(Testimado, 'TIEMPO ESTIMADO DE DURACIÓN (HORAS):')

    if index != -1:
      Testimado = Testimado[index + 1]
    else:
      ic(Testimado)
      Testimado = "·"
  except:
    ic({"archivo: ": hoja, "variable: ":texto})
    Testimado = "·"
  return Testimado

In [ ]:
# Test getTestimado using hoja1

Testimado = getTiempoEstimado( hoja1 )
Testimado

'8'

#### (12) Riesgos de Seguridad

Array of Arrays by row

In [ ]:
# Function to getRiesgos to_pandas()

def getRiesgos( hoja ):
    riesgos_dict = {}
    try:
        df_riesgos = hoja.find_tables( clip = bx_riesgos_epps )
        df_riesgos = df_riesgos.tables[0].to_pandas().replace('', np.nan).dropna(how = 'all')
        for _, row in df_riesgos.iterrows():
            key = row['RIESGOS EXISTENTES:']
            value = [row[col] for col in df_riesgos.columns if col != 'RIESGOS EXISTENTES:']
            riesgos_dict[key] = value
    except:
        ic(hoja)
        riesgos_dict = "·"
    return riesgos_dict

In [ ]:
# Test getRiesgos
riesgos = getRiesgos( hoja1 )
riesgos


{'BIOLÓGICOS': ['Bacterias',
  'Enfermedades infectocontagiosas',
  'Alimentarse bien'],
 'ELÉCTRICOS': ['Electrización (Paso energ. x cuerp',
  'Electrocución o electrización',
  'Usar epps'],
 'ERGONÓMICOS': ['Posiciones en el trabajo',
  'Lesiones músculo esqueléticas',
  'Realizar pausas activas'],
 'MECÁNICOS': ['Caída de objetos', 'Golpes en cabeza y cuerpo', 'Usar epps'],
 'PSICOSOCIALES': ['Realciones sociales', 'Estrés', 'Realizar pausas activas']}

#### (14) Medidas de Seguridad

Solo las medidas de Seguridad

In [ ]:
# Function getMedidasSeguridad return pandas dataframe

def getMedidasSeguridad( hoja ):
    try:
        df_medidas_seg = hoja.find_tables( clip = bx_medidas_seg )
        df_medidas_seg = df_medidas_seg.tables[0].to_pandas()
        df_medidas_seg = df_medidas_seg[['0-MEDIDAS','1-ESTADO']].replace('', np.nan).dropna(how = 'all')
        df_medidas_seg = dict(zip(df_medidas_seg['0-MEDIDAS'],df_medidas_seg['1-ESTADO']))

    except:
        ic(hoja)
        df_medidas_seg = "·"
    return df_medidas_seg

In [ ]:
# Test getMedidasSeguridad

medidas_seg = getMedidasSeguridad( hoja1 )
medidas_seg

{'Equipo: Aislado Eléctricamente': 'BUENO',
 'Herramientas Manuales Adecuadas': 'BUENO',
 'Identificada Área de Seguridad': 'BUENO',
 'Medio de transporte': 'BUENO'}

**bold text**#### (14) EPPs
Listado de los elementos de prevención.

In [ ]:
# Function getEPPs to_pandas

def getEPPs( hoja ):
    try:
        df_epps = hoja.find_tables( clip = bx_medidas_seg )
        df_epps = df_epps.tables[0].to_pandas()
        df_epps = df_epps[['2-EQUIPOS DE PROTECCIÓN','3-ESTADO']].replace('', np.nan).dropna(how = 'all')
        df_epps = dict(zip(df_epps['2-EQUIPOS DE PROTECCIÓN'],df_epps['3-ESTADO']))

    except:
        ic(hoja)
        df_epps = "·"
    return df_epps

In [ ]:
# Test getEPPs

epps = getEPPs( hoja1 )
epps

{'A. Protección para la Cabeza (Casco con barbiquejo)': 'BUENO',
 'B. Protección Ocular (Gafas de acuerdo al trabajo)': 'BUENO',
 'E. Ropa de Trabajo (Uniforme entragado por EERSSA)': 'BUENO',
 'G. Protección manos (Guantes de acuerdo nivel voltaje)': 'BUENO',
 'H. Protección manos (Guantes de trabajo - cuero nitrilo)': 'BUENO',
 'I. Protección extremidades inferiores (Calzado dieléctrico': 'BUENO',
 'L. Equipo de posicionamiento (Cinturón seguridad)': 'BUENO',
 'M. Andamios y/o escaleras (Herr. trab. en altura)': 'BUENO',
 'M. Herramienta aislada': 'BUENO'}

In [ ]:
# medidas_seg to_pandas()

df_medidas_seg = hoja1.find_tables( clip = bx_medidas_seg )
df_medidas_seg = df_medidas_seg.tables[0].to_pandas()
df_medidas_seg

,0-MEDIDAS,1-ESTADO,2-EQUIPOS DE PROTECCIÓN,3-ESTADO
0,Equipo: Aislado Eléctricamente,BUENO,A. Protección para la Cabeza (Casco con barbiq...,BUENO
1,Herramientas Manuales Adecuadas,BUENO,B. Protección Ocular (Gafas de acuerdo al trab...,BUENO
2,Identificada Área de Seguridad,BUENO,E. Ropa de Trabajo (Uniforme entragado por EER...,BUENO
3,Medio de transporte,BUENO,G. Protección manos (Guantes de acuerdo nivel ...,BUENO
4,,,H. Protección manos (Guantes de trabajo - cuer...,BUENO
5,,,I. Protección extremidades inferiores (Calzado...,BUENO
6,,,L. Equipo de posicionamiento (Cinturón seguridad),BUENO
7,,,M. Andamios y/o escaleras (Herr. trab. en altura),BUENO
8,,,M. Herramienta aislada,BUENO
9,,,,


#### (15) Precauciones

Texto de Precauciones

In [ ]:
# Funcion getPrecauciones using get_text()

def getPrecauciones( hoja ):
    try:
        precauciones = hoja.get_text( clip = bx_precauciones ).strip()
        precauciones = precauciones.replace('PRECAUCIONES:', "").strip()
    except:
        ic({"archivo: ": hoja, "variable: ":precauciones})
        precauciones = "·"
    return precauciones

In [ ]:
# Test getPrecauciones

precauciones = getPrecauciones( hoja1 )
precauciones

'Cumplir con el código de ética de la EERSSA. Delimitar adecuadamente el área de trabajo'

#### (16) Carencias

Texto de carencias, la mayoria de veces será vacio

In [ ]:
# Function getCarencias using get_text()

def getCarencias( hoja ):
    try:
        carencias = hoja.get_text( clip = bx_carencias ).strip()
        carencias = carencias.replace('CARENCIAS:', "").strip()
    except:
        ic({"archivo: ": hoja, "variable: ":carencias})
        carencias = "·"
    return carencias

In [ ]:
# Test getCarencias

carencias = getCarencias( hoja1 )
carencias

''

#### (17) Fecha Inicio Hoja 1

In [ ]:
# Function getFechaFinal using to_pandas

def getFechaInicioHoja1( hoja ):
    try:
        df_fecha_final = hoja.find_tables( clip = bx_firmas )
        df_fecha_final = df_fecha_final.tables[0].to_pandas()
        df_fecha_final = df_fecha_final.iloc[1,0].strip().split(':')
        df_fecha_final = df_fecha_final[1].strip()
    except:
        ic(hoja)
        df_fecha_final = "·"
    return df_fecha_final

In [ ]:
# Test for getFechaFinal

fecha_final = getFechaInicioHoja1( hoja1 )
fecha_final

'jueves, 04 de julio del 2024'

#### (18) Firmas

In [ ]:
# FUnction getFirmas to_pandas()

def getFirmas( hoja ):
    try:
        df_firmas = hoja.find_tables( clip = bx_firmas )
        df_firmas = df_firmas.tables[0].to_pandas()
        df_firmas = df_firmas.iloc[0,:].to_list()
        df_firmas = [ item for item in df_firmas if item != '' ]
        df_firmas = df_firmas[1:]
    except:
        ic(hoja)
        df_firmas = "·"
    return df_firmas

In [ ]:
# Test getFirmas

firmas = getFirmas( hoja1 )
firmas

['MORALES RIVERA LUIS ALBERTO',
 'PALACIOS MERINO ERNESTO VLADIMIR',
 'MARQUEZ APOLO JHONNY FABIAN']

In [ ]:
# firmas to_pandas()

df_firmas = hoja1.find_tables( clip = bx_firmas )
df_firmas = df_firmas.tables[0].to_pandas()
df_firmas

,Col0,Col1,AUTORIZACIONES:,Col3
0,NOMBRE:,MORALES RIVERA LUIS ALBERTO,PALACIOS MERINO ERNESTO VLADIMIR,MARQUEZ APOLO JHONNY FABIAN
1,"FECHA: jueves, 04 de julio del 2024",None,JEFE ZONAL,INGENIERO 1


### Hoja 2

#### (21) Cuadrilla

In [ ]:
# Function getCuadrilla using get_text()

def getCuadrilla( hoja ):
    try:
        cuadrilla = hoja.get_text( clip = bx_cuadrilla ).strip()
    except:
        ic({"archivo: ": hoja, "variable: ":cuadrilla})
        cuadrilla = "·"
    return cuadrilla

In [ ]:
# Test getCuadrilla

cuadrilla = getCuadrilla( hoja2 )
cuadrilla

'Zamora Z1 (Cuadrilla. AP Nro. 4)'

#### (22) Kilometrajes

In [ ]:
# Function getKilometraje using find_tables()

def getKilometraje( hoja ):
    try:
        df_kilometraje = hoja.find_tables( clip = bx_kilometraje )
        df_kilometraje = df_kilometraje.tables[0].header.names
        kmInicio = int(df_kilometraje[2].replace(',',""))
        kmFinal = int(df_kilometraje[4].replace(',',""))
        kmRecorrido = int(df_kilometraje[6].replace(',',""))
    except:
        ic(hoja)
        kmInicio = "·"
        kmFinal = "·"
        kmRecorrido = "·"
    return kmInicio, kmFinal, kmRecorrido

In [ ]:
# Test getKilometraje

kmInicio, kmFinal, kmRecorrido = getKilometraje( hoja2 )
[kmInicio, kmFinal, kmRecorrido]

[53091, 53168, 77]

In [ ]:
# kilometraje to_pandas()

df_kilometraje = hoja2.find_tables( clip = bx_kilometraje )
df_kilometraje = df_kilometraje.tables[0].header.names
df_kilometraje

['KILOMETRAJE:', 'Inicial:', '53,091', 'Final:', '53,168', 'Recorridos:', '77']

#### (23) Actividades

Es necsario convertir las Actividades.

 - Reonmbrar las columnas,
 - Eliminar la primera fila confirmando que su primer item sea != 1

In [ ]:
# Function getActividades to_pandas()

def getActividades( hoja ):
    try:
        df_actividades = hoja.find_tables( clip = bx_actividades, strategy = 'lines_strict' )
        df_actividades = df_actividades.tables[0].to_pandas()
        df_actividades.columns = ['Item','Actividad','Descipcion','Ali','Alimentador','Tipo','FechaInicial','FechaFinal']

        while len(df_actividades.iloc[0,0]) > 2:
            df_actividades = df_actividades.drop(0)
            df_actividades = df_actividades.reset_index(drop=True)

        df_actividades['FechaInicial'] = df_actividades['FechaInicial'].str.replace('\n', ' ')
        df_actividades['FechaFinal']   = df_actividades['FechaFinal'].str.replace('\n', ' ')
        df_actividades = df_actividades.replace('', np.nan).dropna(how='all')
        df_actividades = df_actividades.replace(np.nan, "·" )
        df_actividades = df_actividades.to_dict('records')
    except:
        ic(hoja)
        df_actividades = "·"
    return df_actividades


In [ ]:
# Test getActividades

actividades = getActividades( hoja2 )
actividades

[{'Item': '1',
  'Actividad': 'PROG',
  'Descipcion': 'En la agencia de la EERSSA El Pangui se coordina los trabajos.',
  'Ali': '·',
  'Alimentador': '·',
  'Tipo': 'RUTINARIA',
  'FechaInicial': '2024-07-04 08:00:00',
  'FechaFinal': '2024-07-04 08:15:00'},
 {'Item': '2',
  'Actividad': 'NO PROG',
  'Descipcion': 'Pangui en el centro de la ciudad, calle Quito y Ángel Vélez se ayuda\na la cuadrilla a cargo del compañero Henrry Mendieta a realizar el\ncambio de conductor de la segunda fase de la red de B/T',
  'Ali': 'ALI',
  'Alimentador': 'El Pangui',
  'Tipo': 'CORRECTIVO',
  'FechaInicial': '2024-07-04 08:15:00',
  'FechaFinal': '2024-07-04 10:00:00'},
 {'Item': '3',
  'Actividad': '·',
  'Descipcion': 'En los tramos entre las est. No. 028098, 028097, 028096. MOTIVO:\nempalmes en mal estado y de retensar y ubicar conectores en el cruce\nde las líneas.',
  'Ali': '·',
  'Alimentador': '·',
  'Tipo': '·',
  'FechaInicial': '·',
  'FechaFinal': '·'},
 {'Item': '4',
  'Actividad': 'PRO

In [ ]:
# Actividades
df_id_ot = hoja2.find_tables( clip = bx_actividades, strategy = 'lines_strict' )
df_id_ot = df_id_ot.tables[0].to_pandas()
df_id_ot.columns = ['Item','Actividad','Descipcion','Ali','Alimentador','Tipo','FechaInicial','FechaFinal']
df_id_ot


,Item,Actividad,Descipcion,Ali,Alimentador,Tipo,FechaInicial,FechaFinal
0,A C T I V I D A D E S,None,None,None,None,None,None,None
1,No ACTIVIDAD DESCRIPCIÓN DE LA ACTIVIDAD REALI...,None,None,None,None,None,None,None
2,1,PROG,En la agencia de la EERSSA El Pangui se coordi...,,,RUTINARIA,2024-07-04\n08:00:00,2024-07-04\n08:15:00
3,2,NO PROG,"Pangui en el centro de la ciudad, calle Quito ...",ALI,El Pangui,CORRECTIVO,2024-07-04\n08:15:00,2024-07-04\n10:00:00
4,3,,"En los tramos entre las est. No. 028098, 02809...",,,,,
5,4,PROG,RECLAMO No. 1100629005 03-07-24/12:05. Pangui ...,ALI,El Pangui,CORRECTIVO,2024-07-04\n10:00:00,2024-07-04\n10:47:00
6,5,,Por tal motivo se ocupa materiales de 250 W Na...,,,,,
7,6,,NOTA: la envoltura de la carcasa de esta lumin...,,,,,
8,7,PROG,RECLAMO No. 1100629005 03-07-24/12:05. Pangui ...,ALI,El Pangui,CORRECTIVO,2024-07-04\n10:47:00,2024-07-04\n11:45:00
9,8,,Por tal motivo se ocupa materiales de 250 W Na...,,,,,


#### (24) Observaciones

In [ ]:
# Function to getObservaciones using get_text()

def getObservaciones( hoja ):
    try:
        observaciones = hoja.get_text( clip = bx_observaciones ).strip()
    except:
        ic({"archivo: ": hoja, "variable: ":observaciones})
        observaciones = "·"
    return observaciones

In [ ]:
# Test getObservaciones

observaciones = getObservaciones( hoja2 )
observaciones

''

#### (25) Terminado

In [ ]:
# Function getTerminado using get_text()

def getTerminado( hoja ):
    try:
        terminado = hoja.get_text( clip = bx_terminado ).strip()
    except:
        ic({"archivo: ": hoja, "variable: ":terminado})
        terminado = "·"
    return terminado

In [ ]:
# Test getTerminado

terminado = getTerminado( hoja2 )
terminado

'TERMINADO'

#### (36) Fecha Final Hoja 2

In [ ]:
# Function getFechaFinal using get_text()

def getFechaFinal2( hoja ):
    try:
        fechaFinal2 = hoja.get_text( clip = bx_fechaFin ).strip()
    except:
        ic({"archivo: ": hoja, "variable: ":fechaFinal2})
        fechaFinal2 = "·"
    return fechaFinal2



In [ ]:
# Test getFechaFinal2 using get_text()

fechaFinal2 = getFechaFinal2( hoja2 )
fechaFinal2

'04/07/2024 20:00:00'

#### (37) Accidentes


In [ ]:
# Functio getAccidentes uson get_text()

def getAccidentes( hoja ):
    try:
        accidentes = hoja.get_text( clip = bx_accidentes ).strip()
    except:
        ic({"archivo: ": hoja, "variable: ":accidentes})
        accidentes = "·"
    return accidentes

In [ ]:
# Test getAccidentes

accidentes = getAccidentes( hoja2 )
accidentes

'NO'

### Close the document

In [ ]:
doc.close()
noCar.close()

### Create the JSON Object

Muy similar a ExtraerDatosOT() le doy la ruta a un archivo PDF de tipo Orden de Trabajo y me debe devolver un objeto tipo diccionario.

Hasta el momento, en las filas de actividades se encuentra `nan` cundo alguna celda esta vacia. Esto puede se necesario cambiarlo por `"·"` para compatibilidad con el anterior.

```
""" Libreria para la extracción de datos desde una Orden de Trabajo personal
    Autor: Ernesto Palacios
    Fecha: 14/12/2023

    PyMuPDF

    Pendientes:

    - responsable
    - colaboradores
    - vehiculo
    - propietario
    - chofer
    - TipoTrabajo
"""
```

In [ ]:

def toJSON_ot( pdf_path:str ):

    pdf = None
    OT_dict = {"VERSION" : VERSION}
    OT_dict = OT_dict | { "link" : pdf_path }

    try:
        with pymupdf.open( pdf_path ) as pdf:

            # Validación del PDF como una Orden de trabajo EERSSA
            if pdf.metadata['producer'] not in ['FPDF 1.7']:
                OT_dict = OT_dict | {"id_ot": np.nan}
                OT_dict = OT_dict | {"exito": False}
                return OT_dict

            # Extracción de los datos usando las funciones get

            OT_dict = OT_dict | { "id_ot"          : getId_Ot( pdf[0] )}
            OT_dict = OT_dict | { "cuadrilla"      : getCuadrilla( pdf[1] )}
            OT_dict = OT_dict | { "responsable"    : getResponsable( pdf[0] )}
            OT_dict = OT_dict | { "colaboradores"  : getColaboradores( pdf[0] )}

            fecha = getFechaInicio( pdf[0] )
            OT_dict = OT_dict | { "diaSemana"      : getDiaSemana( fecha )}
            OT_dict = OT_dict | { "fecha"          : toDateEcuador( fecha ) }
            OT_dict = OT_dict | { "fechaInicio"    : getFechaInicioHoja1( pdf[0] )} # Convertir a DateTimeObject compatible with BSON
            OT_dict = OT_dict | { "fechaFinal"     : getFechaFinal2( pdf[1] )}

            OT_dict = OT_dict | { "sitio"          : getSitio( pdf[0] )}
            OT_dict = OT_dict | { "descripcion"    : getDescripcion( pdf[0] )}
            OT_dict = OT_dict | { "tEstimado"      : getTiempoEstimado( pdf[0] )}
            OT_dict = OT_dict | { "vehiculo"       : getVehiculo( pdf[0] )}

            kmInicial, kmFinal, kmRecorrido = getKilometraje( pdf[1] )
            OT_dict = OT_dict | { "kmInicial"      : kmInicial}
            OT_dict = OT_dict | { "kmFinal"        : kmFinal}
            OT_dict = OT_dict | { "kmRecorrido"    : kmRecorrido}

            OT_dict = OT_dict | { "trabajo"        : getTiposTrabajo( pdf[0] )}
            OT_dict = OT_dict | { "riesgos"        : getRiesgos( pdf[0] )}
            OT_dict = OT_dict | { "seguridad"      : getMedidasSeguridad( pdf[0] )}
            OT_dict = OT_dict | { "epps"           : getEPPs( pdf[0] )}
            OT_dict = OT_dict | { "precauciones"   : getPrecauciones( pdf[0] )}
            OT_dict = OT_dict | { "carencias"      : getCarencias( pdf[0] )}
            OT_dict = OT_dict | { "accidente"      : getAccidentes( pdf[1] )}
            OT_dict = OT_dict | { "observaciones"  : getObservaciones( pdf[1] )}
            OT_dict = OT_dict | { "numeracion"     : getNumeracion( pdf[0] )}
            OT_dict = OT_dict | { "gerencia"       : getGerencia( pdf[0] )}
            OT_dict = OT_dict | { "estado"         : getTerminado( pdf[1] )}
            OT_dict = OT_dict | { "firmas"         : getFirmas( pdf[0] )}
            OT_dict = OT_dict | { "actividades"    : getActividades( pdf[1])}

            if pdf.page_count > 2: # Funciona para OTs con mas de dos hojas
                for x in range(2, pdf.page_count):
                    OT_dict["actividades"].append( getActividades( pdf[x] ) )

            OT_dict = OT_dict | { "exito"          : True }

    except:
        ic(pdf_path)
        OT_dict = OT_dict | { "exito": False }

    return OT_dict

In [ ]:
test = toJSON_ot( '/content/drive/MyDrive/Colab_Disk/OT/2024/01 ENERO/04 CUADRILLA YACUAMBI/Orden de trabajo Yacuambi 22-01-2024 (NL).pdf' )
test

to_pandas(): this method returns the table as a pandas DataFrame.

extract(): this method returns the text content of each table cell as a list of list of strings.






## Test PyMuPDF

In [ ]:

"""
    Listado de todos los archivos con extensión .pdf en la ruta `pdf_dir`

"""

list_pdfs = []

# TEST    ****    TEST   **** TEST
for path in Path( '/content/drive/MyDrive/Colab_Disk/GestionOT/TestDirectories/0_ImportPdfOT' ).glob("**/*.pdf"):
    list_pdfs.append( str(path) )

len(list_pdfs)

27

In [ ]:
test_dir = []
for file in list_pdfs:
    test_dir.append( toJSON_ot( file ) )


ot_pdf| <ipython-input-39-a01a8d2f98f7>:19 in getTiempoEstimado()
         {"archivo: ": hoja, "variable: ":texto}: {'archivo: ': page 0 of /content/drive/MyDrive/Colab_Disk/OT/2024/08 AGOSTO/05 CUADRILLA YANTZAZA/Orden de trabajo Yantzaza 17-08-2024 (AA).pdf,
                                                   'variable: ': ['FECHA DE INICIO:',
                                                                  'sábado, 17 de agosto del 2024',
                                                                  'TIEMPO ESTIMADO DE DURACIÓN (HORAS):']}
ot_pdf| <ipython-input-43-5dfaeef69960>:11 in getMedidasSeguridad()- hoja: page 0 of /content/drive/MyDrive/Colab_Disk/OT/2024/08 AGOSTO/23 (electricistas) Ag. El Pangui/Orden de trabajo El Pangui 07-08-2024 (DG - Electricistas).pdf
ot_pdf| <ipython-input-45-64b430c559d5>:11 in getEPPs()- hoja: page 0 of /content/drive/MyDrive/Colab_Disk/OT/2024/08 AGOSTO/23 (electricistas) Ag. El Pangui/Orden de trabajo El Pangui 07-08-2024 (DG - Electricista

In [ ]:
test_dir[13]

## Listado de OTs para convertir a DataFrame



In [ ]:
"""
    Listado de todos los archivos con extensión .pdf en la ruta `pdf_dir`

"""

list_pdfs = []

# Todos los PDF Alrededor de 20.000 ?
for path in Path( pdf_dir ).glob("**/*.pdf"):
    list_pdfs.append( str(path) )

len(list_pdfs)

328

In [ ]:
ots_raw = []
for pdfFile in list_pdfs:
    ots_raw.append( toJSON_ot( pdfFile ) )

with open( path_dict, 'wb' ) as fp:  # Guardamos el diccionario en formato JSON
    pickle.dump( ots_raw, fp)

df_dask = pd.DataFrame(ots_raw) # Convert to Pandas Dataframe
df_dask.to_pickle( path_raw )       # Guardamos en formato multidimensional el Dataframe

len(df_dask)

ot_pdf| <ipython-input-39-a01a8d2f98f7>:19 in getTiempoEstimado()
         {"archivo: ": hoja, "variable: ":texto}: {'archivo: ': page 0 of /content/drive/MyDrive/Colab_Disk/OT/2024/08 AGOSTO/05 CUADRILLA YANTZAZA/Orden de trabajo Yantzaza 17-08-2024 (AA).pdf,
                                                   'variable: ': ['FECHA DE INICIO:',
                                                                  'sábado, 17 de agosto del 2024',
                                                                  'TIEMPO ESTIMADO DE DURACIÓN (HORAS):']}
ot_pdf| <ipython-input-43-5dfaeef69960>:11 in getMedidasSeguridad()- hoja: page 0 of /content/drive/MyDrive/Colab_Disk/OT/2024/08 AGOSTO/23 (electricistas) Ag. El Pangui/Orden de trabajo El Pangui 07-08-2024 (DG - Electricistas).pdf
ot_pdf| <ipython-input-45-64b430c559d5>:11 in getEPPs()- hoja: page 0 of /content/drive/MyDrive/Colab_Disk/OT/2024/08 AGOSTO/23 (electricistas) Ag. El Pangui/Orden de trabajo El Pangui 07-08-2024 (DG - Electricista

328

In [ ]:
len(ots_raw)

328

In [ ]:
df_dask.sample(5)

,VERSION,link,id_ot,cuadrilla,responsable,colaboradores,diaSemana,fecha,fechaInicio,fechaFinal,...,precauciones,carencias,accidente,observaciones,numeracion,gerencia,estado,firmas,actividades,exito
243,0.10.0,/content/drive/MyDrive/Colab_Disk/OT/2024/08 A...,134996,El Pangui Z1 (Cuadrilla. Nro. 4),"[MENDIETA MENDIETA HENRRY ALEXANDER, JECU]","{'total': 2, 'nombres': [['OCHOA CUEVA JONATHA...",lunes,2024-08-05 00:00:00-05:00,"lunes, 05 de agosto del 2024",05/08/2024 20:14:00,...,Respetar las 5 reglas de oro.,,NO,,157,OPERACIÓN Y MANTENIMIENTO,TERMINADO,"[MENDIETA MENDIETA HENRRY ALEXANDER, PALACIOS ...","[{'Item': '1', 'Actividad': 'PROG', 'Descipcio...",True
122,0.10.0,/content/drive/MyDrive/Colab_Disk/OT/2024/08 A...,134825,Gualaquiza Z1 (Cuadrilla. Nro. 3),"[LITUMA CORDOVA CESAR RUBEN, JECU]","{'total': 2, 'nombres': [['GUAMAN CONTENTO MAR...",viernes,2024-08-02 00:00:00-05:00,"viernes, 02 de agosto del 2024",02/08/2024 18:28:00,...,Cumplir con el código de ética de la EERSSA. C...,FALTA COMPLETAR LA CUADRILLA CON LINIERO 2 Y C...,NO,,194,OPERACIÓN Y MANTENIMIENTO,TERMINADO,"[LITUMA CORDOVA CESAR RUBEN, PALACIOS MERINO E...","[{'Item': '1', 'Actividad': 'PROG', 'Descipcio...",True
176,0.10.0,/content/drive/MyDrive/Colab_Disk/OT/2024/08 A...,134707,Zamora Z1 (Cuadrilla. Nro. 6),"[RIVERA GUAMAN SEGUNDO PATRICIO, JECU]","{'total': 0, 'nombres': []}",jueves,2024-08-01 00:00:00-05:00,"jueves, 01 de agosto del 2024",01/08/2024 17:05:00,...,Cumplir a Cabalidad con la Restricción Laboral...,,NO,,145,OPERACIÓN Y MANTENIMIENTO,TERMINADO,"[RIVERA GUAMAN SEGUNDO PATRICIO, PALACIOS MERI...","[{'Item': '1', 'Actividad': 'PROG', 'Descipcio...",True
219,0.10.0,/content/drive/MyDrive/Colab_Disk/OT/2024/08 A...,136478,Zamora Z1 (Cuadrilla. Nro. 6),"[RIOS RIOS FRANCISCO FERNANDO, LIN3]","{'total': 3, 'nombres': [['JADAN REQUELME MARC...",viernes,2024-08-30 00:00:00-05:00,"viernes, 30 de agosto del 2024",30/08/2024 17:00:00,...,Cumplir con el código de ética de la EERSSA. C...,,NO,,167,OPERACIÓN Y MANTENIMIENTO,TERMINADO,"[RIOS RIOS FRANCISCO FERNANDO, PALACIOS MERINO...","[{'Item': '1', 'Actividad': 'PROG', 'Descipcio...",True
23,0.10.0,/content/drive/MyDrive/Colab_Disk/OT/2024/08 A...,136318,Zamora Z1 (Cuadrilla. AP Nro. 4),"[MORALES RIVERA LUIS ALBERTO, JECU]","{'total': 3, 'nombres': [['CABRERA CABRERA MIG...",miércoles,2024-08-28 00:00:00-05:00,"miércoles, 28 de agosto del 2024",28/08/2024 22:00:00,...,Cumplir con el código de ética de la EERSSA. D...,,NO,,175,OPERACIÓN Y MANTENIMIENTO,TERMINADO,"[MORALES RIVERA LUIS ALBERTO, PALACIOS MERINO ...","[{'Item': '1', 'Actividad': 'PROG', 'Descipcio...",True


# Conexión con MongoDB


## Simple upload

In [ ]:
!pip install pymongo

In [ ]:
from google.colab import userdata
import pymongo
import pickle
import pandas as pd


uri = userdata.get('MONGO_KEY')
client = pymongo.MongoClient( uri )

# Ping para confirmar que se ha establecido la conexion
try:
    client.admin.command('ping')
    print("Conexion exitosa!!!!")
except Exception as e:
    print(e)

Conexion exitosa!!!!


In [ ]:
db_eerssa = client.eerssa                      # Base de datos EERSSA
MongoColl_TDD = db_eerssa.ot_TDD   # Coleccion para pruebas de carga de datos version 0.10
MongoColl_ot_v10 = db_eerssa.ot_v10

In [ ]:
# cargar la db_dictionary - Array de JSON Files
path_test = '/content/drive/MyDrive/Colab_Disk/OT/2023_PyMu_Todos_a_dict.pkl'
with open( path_test, 'rb' ) as fp:
    db_dictionary = pickle.load(fp)

len(db_dictionary)

4314

In [ ]:
db_dictionary[1000]

In [ ]:
# Upload each element to db_collection_prueba

for ot in db_dictionary:
    MongoColl_ot_v10.insert_one( ot )

# Exportar kilometrajes

🐞 HAY UN ERROR cuando dos OT tienen la misma Fecha


Lo normal sería utilizar PyMongoDriver para descargar los docuemtos;
Luego convertirlos a Dataframe, que es el paso en el cual nos encontramos

Nueva idea:

creo una nueva columna que calcula el día de la semana en base al indice con DateTimeObject... luego... los días en los que no coincide el día calculado con el día que viene de la OT son los días adicionales que fueron insertados, por lo tanto cambio los elementos que necesito cambiar en esas filas y voilá, listo.


miércoles


In [ ]:
import pandas as pd
import pickle
import locale
from datetime import datetime


path_raw = '/content/drive/MyDrive/Colab_Disk/OT/2024-08_agosto_0_raw.pkl'



def get_day_name_spanish(date_time_obj):
    """
        Translates a DateTimeObject and returns the name of the day in Spanish.

        Parameters:
        date_time_obj (datetime): The input DateTimeObject.

        Returns:
        str: The name of the day in Spanish.
    """

    day_name = date_time_obj.strftime("%A").lower()

    day_mapping = {
        'monday': 'lunes',
        'tuesday': 'martes',
        'wednesday': 'miércoles',
        'thursday': 'jueves',
        'friday': 'viernes',
        'saturday': 'sábado',
        'sunday': 'domingo'
    }

    if day_name in day_mapping:
        day_name = day_mapping[day_name]
    else:
        day_name = None

    return day_name

df = pd.read_pickle( path_raw )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype                            
---  ------         --------------  -----                            
 0   VERSION        328 non-null    object                           
 1   link           328 non-null    object                           
 2   id_ot          328 non-null    int64                            
 3   cuadrilla      328 non-null    object                           
 4   responsable    328 non-null    object                           
 5   colaboradores  328 non-null    object                           
 6   diaSemana      328 non-null    object                           
 7   fecha          328 non-null    datetime64[ns, America/Guayaquil]
 8   fechaInicio    328 non-null    object                           
 9   fechaFinal     328 non-null    object                           
 10  sitio          328 non-null    object             

In [ ]:
sin_exito = df[df.exito == False]
sin_exito

,VERSION,link,id_ot,cuadrilla,responsable,colaboradores,diaSemana,fecha,fechaInicio,fechaFinal,...,precauciones,carencias,accidente,observaciones,numeracion,gerencia,estado,firmas,actividades,exito


In [ ]:
# PRE - PROCESAMIENTO para elaborar Informe de Kilometrajes.

# Descargo el archivo correspondiente definida previamente en la variable path_raw
df = pd.read_pickle( path_raw )

# Expongo el número del vehículo como una columna.
df['nro_vehiculo'] = df['vehiculo'].apply(lambda x: x.get('numero') if isinstance(x, dict) else None)

# Filtro las columnas relevantes al reporte de kilometrajes.
df = df[['fecha','diaSemana','fechaFinal','sitio','kmInicial','kmFinal','kmRecorrido','nro_vehiculo']]

# Elimino las filas en las cuales nro_vehiculo == "·"
df = df[df.nro_vehiculo != "·"]


df

,fecha,diaSemana,fechaFinal,sitio,kmInicial,kmFinal,kmRecorrido,nro_vehiculo
0,2024-08-01 00:00:00-05:00,jueves,01/08/2024 17:05:00,"Pachicutza, San Francisco El Padmi, Los Encuen...",88049,88224,175,R-159
1,2024-08-02 00:00:00-05:00,viernes,02/08/2024 17:15:00,"Zamora, San Francisco de La Quebrada de Cumbar...",88230,88302,72,R-159
2,2024-08-05 00:00:00-05:00,lunes,05/08/2024 17:05:00,"Zamora, Timbara, Cumbaratza",88405,88565,160,R-159
5,2024-08-08 00:00:00-05:00,jueves,08/08/2024 20:10:00,Zamora,54367,54413,46,2-91
6,2024-08-09 00:00:00-05:00,viernes,09/08/2024 12:41:00,Namirez Bajo,120616,120671,55,2-110
...,...,...,...,...,...,...,...,...
322,2024-08-23 00:00:00-05:00,viernes,23/08/2024 17:05:00,"El Pangui, El Guismi, Pachicutza, Tundayme, Sa...",24840,24879,39,4-118
324,2024-08-26 00:00:00-05:00,lunes,26/08/2024 18:01:00,"Pakintza, Machinaza alto, Machinaza Bajo, Guis...",24879,24963,84,4-118
325,2024-08-27 00:00:00-05:00,martes,27/08/2024 18:10:00,"El Pangui, El Guismi, San Gabriel, El Miasi, E...",24963,25062,99,4-118
326,2024-08-28 00:00:00-05:00,miércoles,28/08/2024 17:00:00,"El Pangui, El Guismi, Quimi, Machinza, San Roq...",25062,25107,45,4-118


In [ ]:
print(df['nro_vehiculo'].unique())

['R-159' '2-91' '2-110' 'R-126' '2-61' '2-112' '4-62' 'R-171' '4-52'
 '4-36' '2-21' '2-102' '2-45' '4-100' '4-118']


- Agrupar por cada tipo de vehiculo,
- rellenar fechas faltantes,

In [ ]:
df_zmra = (df[df.nro_vehiculo == 'R-126']
           .sort_values(by='fecha')
          )

df_zmra['diaFecha'] = df_zmra['fecha'].apply(lambda x: get_day_name_spanish(x))

diasDescanso = df_zmra['diaSemana'] != df_zmra['diaFecha']

df_zmra.loc[diasDescanso,'sitio'] = 'NO HAY RECORRIDO EN ESTE DÍA'
df_zmra.loc[diasDescanso,'fechaFinal'] = 'Descanso'

df_zmra[['fecha','diaFecha','kmInicial','kmFinal','kmRecorrido','sitio', 'kmFinal', 'nro_vehiculo']]


,fecha,diaFecha,kmInicial,kmFinal,kmRecorrido,sitio,kmFinal,nro_vehiculo
26,2024-08-01 00:00:00-05:00,jueves,263620,263690,70,"Nuevo Quito, Conguime",263690,R-126
27,2024-08-02 00:00:00-05:00,viernes,263789,263889,100,"Yawi,Nuevo Paraiso",263889,R-126
28,2024-08-03 00:00:00-05:00,sábado,263890,263942,52,"Natenza, Guayzimi",263942,R-126
29,2024-08-04 00:00:00-05:00,domingo,263942,264022,80,Paquisha Bellavista,264022,R-126
30,2024-08-05 00:00:00-05:00,lunes,264026,264115,89,"La Libertad, Conguime",264115,R-126
31,2024-08-06 00:00:00-05:00,martes,264119,264200,81,"Chinapintza, La Libertad",264200,R-126
32,2024-08-07 00:00:00-05:00,miércoles,264204,264274,70,"Conguime, La Libertad",264274,R-126
33,2024-08-08 00:00:00-05:00,jueves,264278,264340,62,"Nuevo Paraiso, Conguime",264340,R-126
34,2024-08-09 00:00:00-05:00,viernes,264344,264414,70,Shaime Nuevo Paraíso Heroes Del Cóndor,264414,R-126
35,2024-08-10 00:00:00-05:00,sábado,264418,264488,70,Shaime Nuevo Paraíso Heroes Del Cóndor,264488,R-126


In [ ]:
df_zmra = (df[df.nro_vehiculo == 'R-126']
           .sort_values(by='fecha')
           .set_index('fecha')
           .resample('D')
           .ffill()
           .reset_index()
          )

df_zmra['diaFecha'] = df_zmra['fecha'].apply(lambda x: get_day_name_spanish(x))

diasDescanso = df_zmra['diaSemana'] != df_zmra['diaFecha']

df_zmra.loc[diasDescanso,'sitio'] = 'NO HAY RECORRIDO EN ESTE DÍA'
df_zmra.loc[diasDescanso,'fechaFinal'] = 'Descanso'

df_zmra[['fecha','diaFecha','kmInicial','kmRecorrido','sitio', 'kmFinal', 'nro_vehiculo']]

KeyError: 'fecha'

**Modelo de como se debe llenar**

| Dias | Kilometraje Inicial | Recorrido | Trabajo | Lugar |
|------|---------------------|-----------|---------|-------|
|Lunes | 256,485  | 89 | ATENCION DE DAÑOS | Zamora, El Limon |
|Martes| 256,585  | 100| ATENCION DE DAÑOS | Zamora, El Limon |

etc, etc

- Caminar la columna de día, arriba hacia abajo, si se repite el día borrar cambiar la nota de Lugar por: "EL VEHÍCULO PERMANECE EN AGENCIA"

- También cambio el nombre del día y coloco "Descanso"

Nota: puede haber uno o más días repetidos...




---



In [ ]:

# Carga de archivos, todo 2023
import pandas as pd

# Lista de dictionaries
dict_1823 = pd.read_pickle( "/content/drive/MyDrive/Colab_Disk/OT/v081_18_23_dict_array.pkl" )
df_dask = pd.DataFrame( dict_1823 )
df_dask.info()


"""
   Exportar los kilometros recorridos de acuerdo a cada id_ot
"""

df_recorridos = df_dask[[ 'id_ot', 'kmInicial', 'kmFinal','fecha' ]]

def dia_de_fecha_larga( fecha_larga ):
    try:
        dia = fecha_larga.split(',')[0]

    except:
        dia = '·'

    return dia

def anio_de_fecha_larga( fecha_larga ):
    try:
        anio = fecha_larga.split(' ')[-1]

    except:

        anio = '1900'

    return anio


def km_str_to_int( odometro ):

    try:
        respuesta = int(odometro.replace(',',''))

    except:
        respuesta = 0

    return respuesta

def id_no_comas ( id ):
    try:
        respuesta = id.replace(',' , '')
    except:
        respuesta = "0"

    return respuesta

fcha_test = "jueves, 07 de marzo del 2019"
print( dia_de_fecha_larga( fcha_test ) )
print( anio_de_fecha_larga( fcha_test) )
print( km_str_to_int( '·' ) )
print( km_str_to_int( '99,123' ) )

df_recorridos['dia'] = df_recorridos['fecha'].apply( lambda x: dia_de_fecha_larga(x) )
df_recorridos['anio'] = df_recorridos['fecha'].apply( lambda x: anio_de_fecha_larga(x) )
df_recorridos['odo_inicial'] = df_recorridos['kmInicial'].apply( lambda x: km_str_to_int(x) )
df_recorridos['odo_final'] = df_recorridos['kmFinal'].apply( lambda x: km_str_to_int(x) )
df_recorridos['kmRecorridos'] = df_recorridos['odo_final'] - df_recorridos['odo_inicial']

df_recorridos['id_ot'] = df_recorridos['id_ot'].apply( lambda x: id_no_comas(x) )

"""
   Exportar los kilometros recorridos de acuerdo a cada id_ot del año 2023
"""

df_recorridos_2023 = df_recorridos[ df_recorridos.anio == '2023']

##### Sample the Dataframe

df_recorridos_2023.sample(5)

df_recorridos_2023.info()

df_recorridos_2023['kmRecorridos'].describe()

# Ajuste de valores erroneos
df_recorridos_2023.loc[15718,'kmRecorridos'] = 31

df_recorridos_2023

# Exportar a Excel.

df_recorridos_2023.to_excel('kilometrajes_DICIEMBRE_2023_for_R.xlsx')

#### Una vez se tiene las columnas de fechaInicio y fechaFin y tiempo

matriz_actividades.iloc[1095]

#### Opcional, cargar archivo Pickle desde memoria   🔙  DESDE AQUI


# Usando DASK convierte los PDF a Dict y Dataframe

Genera muchos errores, q no tienen mucho sentido, hay q buscar la forma, teoréticamente es posible, pero hay q trabajarlo más.

In [ ]:
"""
    DASK || Local Parallel
    A partir de una lista con las rutas completas de los archivos.
    Utiliza la librería DASK para paralelizar en los núcelos del sistema
    la ejecución.

    Utiliza tabula-py la cual es un wrapper para JVM y utiliza JPipe
"""
ots_raw = []

for pdf in list_pdfs :
    ots_raw.append(delayed(toJSON_ot)(pdf))

dc_resultado = dask.compute(ots_raw, scheduler='processes')
df_dask = pd.DataFrame(dc_resultado[0])

#    Guarda a disco la variable obtenida.

df_dask.to_pickle( path_raw )       # Guardamos en formato multidimensional el Dataframe

with open( path_dict, 'wb' ) as fp:  # Guardamos el diccionario en formato JSON
    pickle.dump( dc_resultado[0], fp)


len(df_dask)

# 2 - REVISIÓN Y CORRECCIÓN
Una vez contamos con el PDF en formato de Dataframe y JSON (Dictionary) tenemos dos caminos

## 2.1 Calificación 🛠 (falta implementar)
Aquí revisamos la OT y resaltamos y generamos la primera parte del LOG.txt, la intención es que se pueda corregir la OT antes de finalizarla,


## 2.2 Creación de Matriz de Actividades
Convertimos el Dataframe multidimensional a un formato de Matriz (2D) Filas-Columnas, durante la transformación se ejecutan correcciones más frecuentes, fechas, horas, con la intención de que la matriz sea lo más correcta posible. Los cambios realizados serán registrados en la segunda sección de LOG.txt.

Se transforma el texto expandiendo las abreviaciones y errores comunes de escritura.

A partir de esta matriz se generará los Informes y se podra Exportar/Importar a Excel para la revisión manual y actualización de OT.

In [ ]:
"""
    Cargamos el Dataset de OT's desde el directorio de trabajo.

    path_raw = " "    # En caso de necesitar cargar un archivo  ... _0_raw.pkl   diferente
                      # Es mejor configurarlo en la primera celda del Notebook
"""

!pip install Unidecode

import pandas as pd
import math
import pickle
import uuid
import numpy as np
from   numpy import nan

from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

from unidecode import unidecode

from pprint import pprint
import re

nltk.download('stopwords')
nltk.download('punkt')
stopword_es = nltk.corpus.stopwords.words('spanish')




df_dask = pd.read_pickle( path_raw )  # Cargamos en formato multidimensional el Dataframe




### Definiciones de funciones para crear la Matriz de Actividades

#### organizarActidades

In [ ]:
def organizarActividades( ot_df ):

    actividades = pd.DataFrame( ot_df['actividades'] )
    actividades = actividades[actividades.evento.notnull()]

    if len(actividades) == 0:
        return actividades


    actividades = actividades.reset_index()

    """····························································
            Validar y Corregir Fecha y Hora.
       ·························································"""


    fInicio = actividades[['item','fechaInicio','fechaFin']].dropna()

    try:

        fInicio['solofechaI'] = fInicio['fechaInicio'].apply( lambda x: re.findall( '\d{4}-\d{2}-\d{2}', x)[0])
        fInicio['solofechaF'] = fInicio['fechaInicio'].apply( lambda x: re.findall( '\d{4}-\d{2}-\d{2}', x)[0])

        fechaModa = fInicio.solofechaI.mode().values[0] # fecha 'moda' en el arreglo

    except:
        # Si en ningun lado existe una Fecha, toma la Fecha final, (si existe)
        if( ot_df['fechaFinal'] != "·" ):
            fechaModa = ot_df['fechaFinal'].split()[0]

        else:
            fechaModa = '16/09/1988' # no es posible obtener una fecha.


    """····························································
            Etiquetar eventos 'LABORA'
       ·························································"""

    etiq_labora = actividades.query( 'actividad.isnull()', engine = 'python' )

    try:

        #  |SE LAB___  // mayusculas o minusculas
        etiq_SE_LABORA = etiq_labora.evento.str.contains('^se lab', case = False, regex = True )
        index_labora = etiq_SE_LABORA[etiq_SE_LABORA].index.values

        for fila in index_labora:
            actividades.at[fila,'actividad'] = 'LABORA'
            actividades.at[fila,'fechaInicio'] = fechaModa + " 00:00:01"
            actividades.at[fila,'fechaFin'] = fechaModa + " 00:00:02"


        #  |Labora_  // mayusculas o minusculas
        etiq_LABORA = etiq_labora.evento.str.contains('^labora ', case = False, regex = True )
        index_labora = etiq_LABORA[etiq_LABORA].index.values

        for fila in index_labora:
            actividades.at[fila,'actividad'] = 'LABORA'
            actividades.at[fila,'fechaInicio'] = fechaModa + " 00:00:01"
            actividades.at[fila,'fechaFin'] = fechaModa + " 00:00:02"

    except:
        index_labora = None



    """····························································
            Etiquetar eventos 'INFO'
       ·························································"""

    indexIni = 0


    # ¿Es el primer Item Informativo. ej DIA FESTIVO?.
    if( type(actividades['fechaInicio'][0]) is float
            and   type(actividades['fechaFin'][0]) is float):

        actividades.at[indexIni,'actividad'] = 'INFO'
        actividades.at[indexIni,'fechaInicio'] = fechaModa + " 00:00:01"
        actividades.at[indexIni,'fechaFin'] = fechaModa + " 00:00:02"




    """····························································
            Organización de Fechas y Hora 'YQ'

       En el caso de existir "huecos" en las actividades, se las
       rellena con la siguiente válida, sea Inicial o Final, no
       importa, se toma la siguiente válida y se la llena
       ·························································"""


    #   | -- | o  |  =>   |  o  |  o  |
    #   caso especial, primera fila sin Hora Inicial. Se duplica y dt = 0
    if( type(actividades['fechaInicio'][0]) is float
            and not (type(actividades['fechaFin'][0]) is float) ):
        actividades.at['fechaInicio'][0] = actividades.at['fechaFin'][0]

    #   |  o | o  |  =>   |  o  |  o  |
    #   | -- | o  |  =>   |  o  |  o  |

    ulitmaFechaCorrecta = None
    filaDondeFalta = None
    fechaQueFalta = None

    for fila in actividades.index:

        # | o | o |   Ambas fechas correctas
        if( not type(actividades['fechaInicio'][fila]) is float
            and not (type(actividades['fechaFin'][fila]) is float) ):

            ulitmaFechaCorrecta = actividades['fechaFin'][fila]

            if( filaDondeFalta != None  ): # Hay un hueco por llenar

                if( fechaQueFalta == 'fechaInicio' ):
                    actividades.at[filaDondeFalta,'fechaInicio'] = actividades['fechaFin'][fila]
                elif( fechaQueFalta == 'fechaFin' ):
                    actividades.at[filaDondeFalta,'fechaFin'] = actividades['fechaInicio'][fila]

                filaDondeFalta = None
                fechaQueFalta = None

        # | - | o |
        if( type(actividades['fechaInicio'][fila]) is float
            and not (type(actividades['fechaFin'][fila]) is float) ):

            ulitmaFechaCorrecta = actividades['fechaFin'][fila]

            if( filaDondeFalta == None ): # No hay huecos previos, reportar este como Hueco
                filaDondeFalta = fila
                fechaQueFalta = 'fechaInicio'
            else:                       # Mover la fechaFinal a la que falta

                actividades.at[filaDondeFalta,fechaQueFalta]= actividades['fechaFin'][fila]

                actividades.at[fila,'fechaFin'] = float('NaN')
                filaDondeFalta = None
                fechaQueFalta = None

        # | o | - |
        if( not type(actividades['fechaInicio'][fila]) is float
            and (type(actividades['fechaFin'][fila]) is float) ):

            ulitmaFechaCorrecta = actividades['fechaInicio'][fila]

            if( filaDondeFalta == None ): # No hay huecos previos, reportar este como Hueco
                filaDondeFalta = fila
                fechaQueFalta = 'fechaFin'
            else:                       # Mover la fechaInicial a la que falta

                actividades.at[filaDondeFalta,fechaQueFalta] = actividades['fechaInicio'][fila]
                actividades.at[fila,'fechaInicio'] = float('NaN')
                filaDondeFalta = None
                fechaQueFalta = None


    """····························································
            Consolidar Items adicionales en uno solo
       ·························································"""
    backup = actividades

    try:

        for iteraciones in range( actividades['fechaInicio'].isna().sum() ):
        #while( actividades['fechaInicio'].isnull().values.any() ):

            fila = 1
            totalIdx = len(actividades.index)

            while fila < totalIdx:
                if( type(actividades.at[actividades.index[fila],'fechaInicio']) is float ):  # AND or OR ?
                    data = actividades.at[ actividades.index[fila-1],'evento' ]
                    actividades.at[ actividades.index[fila-1],'evento' ] = data + '\r' + actividades.at[ actividades.index[fila],'evento' ]
                    actividades = actividades.drop(actividades.index[fila])
                    fila = totalIdx
                fila = fila + 1

            """ Lo programo de esta manera ya que al hacer el Drop no se actualizan los indices
                si uso un lazo FOR. Al hacerlo de esta manera cada vez leo los indices tomando
                en cuenta aquellos que ya elimine. y funciona.
            """

        actividades.item = actividades.item.astype(int)

    except:

         actividades = backup


    return actividades

#### ConvertirOT_a_ActividadesCSV

In [ ]:
def ConvertirOT_a_ActividadesCSV( ot_df ):

    """ Convierte una fila del Dataframe que contiene las OTs y
        devuelve un nuevo dataframe exponienda cada una de las
        actividades de acuerdo al formato descrito.
    """

    if ot_df['exito'] != True:
        return False

    if ot_df['actividades'] == "·":
        return False

    # campos generales a cada una de las actividad
    cuadrilla     = ot_df['cuadrilla']
    primario      = False    # Se obtendrá del SIG (Sistema de Info Geograf)
    horasExtra    = False    # Actividad fuera de horario normal
    desconexion   = False    # ML
    id_ot         = ot_df['id_ot']
    responsable   = ot_df['responsable']
    colaboradores = ot_df['colaboradores']
    vehiculo      = ot_df['vehiculo']
    sitio         = ot_df['sitio']
    dia           = ot_df['fecha']  # solo el nombre del dia de labores: lunes, martes, ...
    fecha         = ot_df['fecha']  # Convertir a Python Time Object
    archivo       = ot_df['link']   # solo el nombre de archivo PDF

    # obtiene el día de la semana: lunes, martes, ....
    try:
        dia = dia.split(',')[0]
    except:
        dia = "·"

    # Convierte 'fecha' a objeto de datetime en Pandas
    try:
        fecha = fecha.split(' ')
        mes = fecha[3]
        dict_mes = {'enero':'01','febrero':'02','marzo':'03','abril':'04',
                    'mayo':'05','junio':'06','julio':'07','agosto':'08',
                    'septiembre':'09','octubre':'10','noviembre':'11','diciembre':'12'}
        for nombre in dict_mes.keys():
            mes = mes.replace(nombre, dict_mes[nombre])

        fecha = fecha[5] +'-'+ mes + '-' + fecha[1]
    except:
        fecha = "·"

    # Obtiene nombre del archivo a PDF
    try:
        archivo = basename(archivo)
    except:
        archivo = "·"

    # Obtener las Actividades

    actividades = organizarActividades( ot_df )
    if len(actividades) == 0:
        return



    if len(colaboradores) == 0:
        colaboradores = "·"
    else:
        colaboradores = ';'.join(colaboradores)

    """
        Se añaden las Columnas con los valores comunes a cada fila de actividad

        VALIDAR UNA MEJOR FORMA DE HACER ESTO EN PANDAS PARA EVITAR ERRORES CUANDO
        SE EJECUTA EN UN GRAN CANTIDAD DE OTS

    """

    actividades['uuid'] = actividades.apply(lambda x: uuid.uuid4(), axis=1)
    actividades.insert( 1, 'cuenta'        , "·" )
    actividades.insert( 2, 'confianza'     , 0.0 )
    actividades.insert( 3, 'cuadrilla'     , cuadrilla  )
    actividades.insert( 4, 'primario'      , primario   )
    actividades.insert( 5, 'horasExtra'    , horasExtra )
    actividades.insert( 8, 'desconexion'   , desconexion)
    actividades.insert( 9, 'id_ot'         , id_ot      )
    actividades.insert( 1, 'responsable'   , responsable)
    actividades.insert( 2, 'colaboradores' , colaboradores )
    actividades.insert( 3, 'vehiculo'      , vehiculo   )
    actividades.insert( 4, 'sitio'         , sitio      )
    actividades.insert( 5, 'dia'           , dia        )
    actividades.insert( 6, 'fecha'         , fecha      )
    actividades.insert( 7, 'archivo'       , archivo    )

    """
        Se reorganizan las columanas
    """

    actividades = actividades[['uuid','item','confianza','cuenta',
                                'evento','redes','alimentador',
                                'tipo','actividad','cuadrilla',
                                'primario','fechaInicio','fechaFin',
                                'horasExtra','desconexion',
                                'responsable','colaboradores',
                                'vehiculo','sitio',
                                'dia','fecha','id_ot','archivo'
                                ]]


    # cambiar los valores para mejorar la lectura y visualización
    actividades = actividades.replace( np.nan, "·", regex=True )

    return( actividades )

### Funcion de convresión a Matriz de actividad

In [ ]:
"""
    Convertir los DataFrame de OT's a Matriz de Actividades

"""
matriz_actividades = pd.DataFrame()

for i in range( (len(df_dask) ) ):

    try:
        nueva = ConvertirOT_a_ActividadesCSV( df_dask.iloc[ i ] )

        if isinstance(nueva, pd.DataFrame):
            matriz_actividades = pd.concat( [ matriz_actividades, nueva ], ignore_index=True)

    except:
        print( f"Error en el Indice: {i}" )


matriz_actividades = matriz_actividades[['uuid','item','confianza','cuenta',
                                'evento','redes','alimentador',
                                'tipo','actividad','cuadrilla',
                                'primario','fechaInicio','fechaFin',
                                'horasExtra','desconexion',
                                'responsable','colaboradores',
                                'vehiculo','sitio',
                                'dia','fecha','id_ot','archivo'
                                ]]

# Guardar archivo Pickle a disco
matriz_actividades.to_pickle( path_matriz )

len(matriz_actividades)

<ipython-input-6-cd9f49ce2ba4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fInicio['solofechaI'] = fInicio['fechaInicio'].apply( lambda x: re.findall( '\d{4}-\d{2}-\d{2}', x)[0])
<ipython-input-6-cd9f49ce2ba4>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fInicio['solofechaF'] = fInicio['fechaInicio'].apply( lambda x: re.findall( '\d{4}-\d{2}-\d{2}', x)[0])
<ipython-input-6-cd9f49ce2ba4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

3236

### Explorar matriz de actividades

In [ ]:
matriz_actividades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3236 entries, 0 to 3235
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   uuid           3236 non-null   object 
 1   item           3236 non-null   int64  
 2   confianza      3236 non-null   float64
 3   cuenta         3236 non-null   object 
 4   evento         3236 non-null   object 
 5   redes          3236 non-null   object 
 6   alimentador    3236 non-null   object 
 7   tipo           3236 non-null   object 
 8   actividad      3236 non-null   object 
 9   cuadrilla      3236 non-null   object 
 10  primario       3236 non-null   bool   
 11  fechaInicio    3236 non-null   object 
 12  fechaFin       3236 non-null   object 
 13  horasExtra     3236 non-null   bool   
 14  desconexion    3236 non-null   bool   
 15  responsable    3236 non-null   object 
 16  colaboradores  3236 non-null   object 
 17  vehiculo       3236 non-null   object 
 18  sitio   

In [ ]:
matriz_actividades['responsable'].unique()  # Nombres de las personas que han elaborado Ordenes de Trabajo

array(['LOZANO SIGCHO NAUN ENRIQUE', 'CABRERA GONZALEZ WILMER DAVID',
       'ALEJANDRO PACHAR AGUSTIN EDUARDO',
       'BARRAZUETA GONZAGA SERVIO GUILLERMO',
       'OCHOA JARAMILLO ANGEL CLAUDIO', 'MACAS CURIPOMA RAMIRO HOMERO',
       'QUIROGA ORDONEZ CARLOS HERNAN',
       'MENDIETA MENDIETA HENRRY ALEXANDER', 'VERA SANCHEZ LIVIO STALIN',
       'LITUMA CORDOVA CESAR RUBEN', 'GUZMAN BARROS MARCO FERNANDO',
       'YAURE DIAZ RAMIRO GABRIEL', 'MORALES RIVERA LUIS ALBERTO',
       'JARA NARVAEZ GALO SILVERIO', 'CORONEL TORRES RUSBEL ISRAEL',
       'BUELE UYAGUARI CESAR CRISTIAN', 'AMBULUDI SILVA FAUSTO JOEL',
       'POMA GUAMAN LUIS ANTONIO', 'CUADRADO LEON RAUL EFREN',
       'CHAMBA CANGO PEDRO ROSALINO', 'SILVA ARMIJOS ROMEL EDUARDO',
       'RIVERA GUAMAN SEGUNDO PATRICIO', 'RIOS RIOS FRANCISCO FERNANDO',
       'LEON CUEVA LUIS ALBERTO', 'PALACIOS MERINO ERNESTO VLADIMIR'],
      dtype=object)

In [ ]:
matriz_actividades.query( " fecha == '·' " )[['id_ot','responsable','evento','fechaInicio','fechaFin']]  # OTS sin fecha inicial - se deberian corregir

,id_ot,responsable,evento,fechaInicio,fechaFin


In [ ]:
matriz_actividades.query( " actividad == '·' " )[['id_ot','responsable','fecha','evento','fechaInicio','fechaFin']]  # OTS sin actividad - solamente informativas

,id_ot,responsable,fecha,evento,fechaInicio,fechaFin
1437,"122,930",YAURE DIAZ RAMIRO GABRIEL,2024-01-25,"En El Pangui se prepara material, se equipa ve...",2024-01-25 08:00:00,2024-01-25 08:10:00
1458,"122,886",YAURE DIAZ RAMIRO GABRIEL,2024-01-24,"En el Pangui se prepara material, se equipa ve...",2024-01-24 08:00:00,2024-01-24 08:10:00
1476,"122,779",YAURE DIAZ RAMIRO GABRIEL,2024-01-23,"En el Pangui se prepara material, se equipa ve...",2024-01-23 08:00:00,2024-01-23 08:05:00
1508,"122,648",MORALES RIVERA LUIS ALBERTO,2024-01-20,En el sector de Chamico se espera para confirm...,2024-01-20 14:00:00,2024-01-20 15:15:00
1509,"122,648",MORALES RIVERA LUIS ALBERTO,2024-01-20,Alimentación y traslado de Chamico hacia Zamora.,2024-01-20 15:15:00,2024-01-20 16:06:00
1582,"121,820",YAURE DIAZ RAMIRO GABRIEL,2024-01-07,En Agencia Zamora se equipa vehículo de herram...,2024-01-07 15:53:00,2024-01-07 16:05:00
1613,"121,774",YAURE DIAZ RAMIRO GABRIEL,2024-01-05,En Agencia Zamora se equipa vehículo con herra...,2024-01-05 17:45:00,2024-01-05 17:55:00
1693,"121,694",MORALES RIVERA LUIS ALBERTO,2024-01-05,"8 luminarias de 150 W Na de 1NP, transformador...",2024-01-05 17:10:00,2024-01-05 17:05:00
1734,"123,009",YAURE DIAZ RAMIRO GABRIEL,2024-01-26,"En Agencia Yantzaza se coordina trabajos, se r...",2024-01-26 08:00:00,2024-01-26 08:20:00


In [ ]:
matriz_actividades.query( " fechaInicio == '·' " )[['id_ot','responsable','evento','fechaInicio','fechaFin']]   # Actividades sin fecha de Inicio, se debería corregir.

,id_ot,responsable,evento,fechaInicio,fechaFin


## 2.3 Correción de OT  ⏪  continuar aqui

Pendientes:

* Las Conversiones de texto deben estar establecidas en un documento de Google Sheets que será una peuqueña 'base de datos' que se ira actualizando y mejorando.

#### limpiar_texto_actividad

Limpia el texto para calcular la actividad, pero no modifica el Dataset

In [ ]:
def limpiar_texto_actividad( actividad ):

    # Operaciones
    txt = unidecode(actividad)  # Quitar tildes y caracteres especiales
    txt = txt.lower()           # todo a minusculas
    txt = re.sub( r'[^\w\s]',' ', txt ) # Eliminar puntuacion


    """
        Definición de palabras a reemplzarzar
        Esta lista se irá ampliando de manera iterativa.

        a futuro cambiar esta lista por un GoogleSheet con las palabras
        que se desee cambiar
    """
    palabras = {
            "#"             : "nro",
            "est"           : "estructura",
            "est."          : "estructura",
            "est."          : "estructura",
            "estructuras"   : "estructura",
            "poste"         : "estructura",
            "tiraf"         : "tirafusible",
            "med."          : "medidor",
            "med"           : "medidor",
            "med"           : "medidor",
            "cc"            : "Centro de Control",
            "trafo"         : "transformador",
            "tranfo"        : "transformador",
            "breiker"       : "breaker",
            "  "            : " ",
        }

    keys = (re.escape(k) for k in palabras.keys())
    pattern = re.compile(r'\b(' + '|'.join(keys) + r')\b')

    resultado = pattern.sub(lambda x: palabras[x.group()], txt)

    return resultado


"""
    Dado el texto de una actividad, devuelve la cuenta a la que pertenece y el nivel de confianza.
"""

def calificar_actividad( actividad ):

    clasificador_actividad_v1 = 'NB_clasif_activ_2024_03.pkl'
    nb_clf = pickle.load(open(clasificador_actividad_v1, 'rb'))

    vect_filename = 'NB_vectorizer.pkl'
    vectorizer = pickle.load(open(vect_filename, 'rb'))

    texto = limpiar_texto_actividad( actividad )

    pred = nb_clf.predict( vectorizer.transform( [texto] ) )

    proba = nb_clf.predict_proba( vectorizer.transform( [texto] ) )

    return ( pred[0] )

#### ordenarFechaStr

In [ ]:
def ordenarFechaStr( texto ):
    try:
        fecha, hora = texto.split(' ')

        if '/' in fecha:
            dia, mes, anio = fecha.split('/')
            respuesta = anio +'-'+ mes +'-'+ dia +' '+ hora
        else:
            respuesta = texto
    except:
        respuesta = texto

    return respuesta

### Cambiar/Mejorar texto de Actividades

### Calificacion de Actividades

In [ ]:
matriz_actividades['cuenta'] = matriz_actividades['evento'].apply( lambda x: calificar_actividad(x) )



---

OLD


#### Limpieza de Dataset - ya ejecutado para convertir StringFechas a DateTime Object

In [ ]:
matriz_actividades['fechaInicio'] = matriz_actividades['fechaInicio'].apply( lambda x: ordenarFechaStr(x))
matriz_actividades['fechaFin'] = matriz_actividades['fechaFin'].apply( lambda x: ordenarFechaStr(x))

In [ ]:
matriz_actividades['fechaInicio'] = pd.to_datetime( matriz_actividades['fechaInicio'], format='%Y-%m-%d %H:%M:%S')
matriz_actividades['fechaFin'] = pd.to_datetime( matriz_actividades['fechaFin'], format='%Y-%m-%d %H:%M:%S')


### exportar kilometrajes para 06MBID

In [ ]:
# Carga de archivos, todo 2023
import pandas as pd

# Lista de dictionaries
dict_1823 = pd.read_pickle( "/content/drive/MyDrive/Colab_Disk/OT/v081_18_23_dict_array.pkl" )
df_dask = pd.DataFrame( dict_1823 )
df_dask.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19256 entries, 0 to 19255
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   link              19256 non-null  object
 1   id_ot             19210 non-null  object
 2   cuadrilla         19209 non-null  object
 3   responsable       19210 non-null  object
 4   responsableFirma  19209 non-null  object
 5   colaboradores     19209 non-null  object
 6   fecha             19209 non-null  object
 7   fechaInicio       19209 non-null  object
 8   fechaFinal        19209 non-null  object
 9   sitio             19209 non-null  object
 10  descripcion       19209 non-null  object
 11  testimado         19209 non-null  object
 12  vehiculo          19209 non-null  object
 13  propietario       19209 non-null  object
 14  chofer            19209 non-null  object
 15  kmInicial         19209 non-null  object
 16  kmFinal           19209 non-null  object
 17  trabajo     

In [ ]:
"""
   Exportar los kilometros recorridos de acuerdo a cada id_ot
"""

df_recorridos = df_dask[[ 'id_ot', 'kmInicial', 'kmFinal','fecha' ]]

In [ ]:
def dia_de_fecha_larga( fecha_larga ):
    try:
        dia = fecha_larga.split(',')[0]

    except:
        dia = '·'

    return dia

def anio_de_fecha_larga( fecha_larga ):
    try:
        anio = fecha_larga.split(' ')[-1]

    except:

        anio = '1900'

    return anio


def km_str_to_int( odometro ):

    try:
        respuesta = int(odometro.replace(',',''))

    except:
        respuesta = 0

    return respuesta

def id_no_comas ( id ):
    try:
        respuesta = id.replace(',' , '')
    except:
        respuesta = "0"

    return respuesta

fcha_test = "jueves, 07 de marzo del 2019"
print( dia_de_fecha_larga( fcha_test ) )
print( anio_de_fecha_larga( fcha_test) )
print( km_str_to_int( '·' ) )
print( km_str_to_int( '99,123' ) )

jueves
2019
0
99123


In [ ]:
df_recorridos['dia'] = df_recorridos['fecha'].apply( lambda x: dia_de_fecha_larga(x) )
df_recorridos['anio'] = df_recorridos['fecha'].apply( lambda x: anio_de_fecha_larga(x) )
df_recorridos['odo_inicial'] = df_recorridos['kmInicial'].apply( lambda x: km_str_to_int(x) )
df_recorridos['odo_final'] = df_recorridos['kmFinal'].apply( lambda x: km_str_to_int(x) )
df_recorridos['kmRecorridos'] = df_recorridos['odo_final'] - df_recorridos['odo_inicial']

df_recorridos['id_ot'] = df_recorridos['id_ot'].apply( lambda x: id_no_comas(x) )

<ipython-input-17-c58018136e9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recorridos['dia'] = df_recorridos['fecha'].apply( lambda x: dia_de_fecha_larga(x) )
<ipython-input-17-c58018136e9b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recorridos['anio'] = df_recorridos['fecha'].apply( lambda x: anio_de_fecha_larga(x) )
<ipython-input-17-c58018136e9b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [ ]:
"""
   Exportar los kilometros recorridos de acuerdo a cada id_ot del año 2023
"""

df_recorridos_2023 = df_recorridos[ df_recorridos.anio == '2023']

##### Sample the Dataframe

In [ ]:
df_recorridos_2023.sample(5)

,id_ot,kmInicial,kmFinal,fecha,dia,anio,odo_inicial,odo_final,kmRecorridos
19158,120704,"98,247","98,436","martes, 19 de diciembre del 2023",martes,2023,98247,98436,189
15353,105955,"56,630","56,664","lunes, 24 de abril del 2023",lunes,2023,56630,56664,34
17706,113035,"266,216","266,283","miércoles, 16 de agosto del 2023",miércoles,2023,266216,266283,67
16987,108488,0,0,"lunes, 05 de junio del 2023",lunes,2023,0,0,0
17846,116671,"327,928","327,954","jueves, 12 de octubre del 2023",jueves,2023,327928,327954,26


In [ ]:
df_recorridos_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4306 entries, 6627 to 19255
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_ot         4306 non-null   object
 1   kmInicial     4306 non-null   object
 2   kmFinal       4306 non-null   object
 3   fecha         4306 non-null   object
 4   dia           4306 non-null   object
 5   anio          4306 non-null   object
 6   odo_inicial   4306 non-null   int64 
 7   odo_final     4306 non-null   int64 
 8   kmRecorridos  4306 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 336.4+ KB


In [ ]:
df_recorridos_2023['kmRecorridos'].describe()

count      4306.000000
mean        119.633767
std        2156.053106
min           0.000000
25%          24.250000
50%          73.000000
75%         109.000000
max      100100.000000
Name: kmRecorridos, dtype: float64

In [ ]:
# Ajuste de valores erroneos
df_recorridos_2023.loc[15718,'kmRecorridos'] = 31

In [ ]:
df_recorridos_2023

,id_ot,kmInicial,kmFinal,fecha,dia,anio,odo_inicial,odo_final,kmRecorridos
6627,sábado 10 de junio del 2023,·,·,"sábado, 10 de junio del 2023",sábado,2023,0,0,0
14942,110420,"264,044","264,128","lunes, 03 de julio del 2023",lunes,2023,264044,264128,84
14943,110454,0,0,"lunes, 03 de julio del 2023",lunes,2023,0,0,0
14944,110448,"264,128","264,213","martes, 04 de julio del 2023",martes,2023,264128,264213,85
14945,110508,"264,213","264,301","miércoles, 05 de julio del 2023",miércoles,2023,264213,264301,88
...,...,...,...,...,...,...,...,...,...
19251,121123,"71,883","71,994","miércoles, 27 de diciembre del 2023",miércoles,2023,71883,71994,111
19252,121196,"71,994","72,019","jueves, 28 de diciembre del 2023",jueves,2023,71994,72019,25
19253,120950,0,0,"lunes, 25 de diciembre del 2023",lunes,2023,0,0,0
19254,121339,0,0,"viernes, 29 de diciembre del 2023",viernes,2023,0,0,0


In [ ]:
# Exportar a Excel.

df_recorridos_2023.to_excel('kilometrajes_DICIEMBRE_2023_for_R.xlsx')

#### Una vez se tiene las columnas de fechaInicio y fechaFin y tiempo

In [ ]:
matriz_actividades.iloc[1095]

uuid                             256d06b7-cb82-40ed-9124-138a594af8d6
item                                                               10
confianza                                                         0.0
cuenta                                                     511.04.002
evento              Zamora barrio Tunantza en la est. No. 179928 s...
redes                                                             ALI
alimentador                                                 Zamora II
tipo                                                       CORRECTIVO
actividad                                                        PROG
cuadrilla                            Zamora Z1 (Cuadrilla. AP Nro. 4)
primario                                                        False
fechaInicio                                       2023-12-04 14:43:00
fechaFin                                          2023-10-04 15:02:00
horasExtra                                                      False
desconexion         

#### Opcional, cargar archivo Pickle desde memoria   🔙  DESDE AQUI


## Exportar a Excel la Matriz de Actividades

**FALTA**:

 - ~~Calificar la actividad~~.
 - ~~Normalizar~~ las fechas a un solo formato, preferible con - en lugar de /
 - Falta corregir las fechas en las actividades con Moda.
 - Arbol de decision para casos específicos. Cuentas especiales
 - Calcular el tiempo en formato H:MM ⏯ se tiene tiempo en formato DateTimeObj

#### Calcular tiempo transcurrido en HH:MM



In [ ]:
matriz_actividades['tiempo'] = (matriz_actividades['fechaFin'] -   matriz_actividades['fechaInicio']).astype({'date': 'datetime64[m]'})

In [ ]:
matriz_actividades.iloc[1000]

uuid                             7dd21104-efcb-480d-8cfb-1fca6f22267c
item                                                                9
confianza                                                         0.0
cuenta                                                              ·
evento              SE LABORA: EP de 08:00 a 13:00 y de 14:00 a 18:21
redes                                                               ·
alimentador                                                         ·
tipo                                                                ·
actividad                                                      LABORA
cuadrilla                                       Jefatura Zonal Zamora
primario                                                        False
fechaInicio                                       2023-12-12 00:00:01
fechaFin                                          2023-12-12 00:00:02
horasExtra                                                      False
desconexion         

In [ ]:
matriz_actividades.sample(3)

,uuid,item,confianza,cuenta,evento,redes,alimentador,tipo,actividad,cuadrilla,...,responsable,colaboradores,vehiculo,sitio,dia,fecha,id_ot,archivo,fechaInicio_time,tiempo
2491,676c8198-686c-4c1f-a807-b06ab49c25e6,3,0.0,511.04.001,"En Shaime, Est 61990 se repone tirafusible de ...",ALI,Paquisha,CORRECTIAS,PROG,Guayzimi Z1 (Cuadrilla. Nro. 7),...,CACAY LUZURIAGA ASDRUBAL HUMBERTO,LARA JIMENEZ JAIME PATRICIO,R-126,"Shaime, Nuevo paraíso",domingo,2023-12-24,"120,946",Orden de trabajo Guayzimi 24-12-2023 (AC).pdf,2023-12-24 10:15:00,0 days 00:25:00
11,4ec59a2a-595b-4988-a5d6-0d4f852ec949,3,0.0,511.05.002,"SE LABORA: RS, LS de 02:20 a 03:38",·,·,·,LABORA,Zamora Z1 (Cuadrilla. Nro. 6),...,SILVA ARMIJOS ROMEL EDUARDO,LEON CUEVA LUIS ALBERTO;SONGOR CARRION LUIS AL...,2-110,"Zamora, Timbara",viernes,2023-12-01,"119,674",Orden de trabajo Zamora 01-12-2023 (RS).pdf,2023-12-01 00:00:01,0 days 00:00:01
1760,929c7ba7-9441-4513-a76b-2ae122e14c09,4,0.0,511.04.001,LUNCH en el mismo sector,·,·,ALIMENTACI,ALIMEN,Yacuambi Z1 (Cuadrilla. Nro. 8),...,CABRERA GONZALEZ WILMER DAVID,ZHUNAULA GUAMAN VICTOR ANIBAL,R-142,"Yacuambi, Nuevo Porvenir",viernes,2023-12-29,"121,268",Orden de trabajo Yacuambi 29-12-2023 (WC).pdf,2023-12-29 12:45:00,0 days 00:45:00


In [ ]:
excel_format = matriz_actividades[[
                    'cuenta',
                    'evento',
                    'alimentador',
                    'tipo',
                    'actividad',
                    'cuadrilla',
                    'primario',
                    'fechaInicio',
                    'fechaFin',
                    'tiempo',
                    'horasExtra',
                    'desconexion',
                    'id_ot',
                    'responsable',
                    'vehiculo',
                    'sitio',
                    'archivo'
                 ]]

In [ ]:
excel_format.to_excel( guardar_Excel, index=False )

In [ ]:
excel_format.sample(5)

,cuenta,evento,alimentador,tipo,actividad,cuadrilla,primario,fechaInicio,fechaFin,tiempo,horasExtra,desconexion,id_ot,responsable,vehiculo,sitio,archivo
1679,511.05.002,"Nos trasladamos a Chamico, Nambija Bajo, Zamor...",·,TRANSPORTE,TRANSP,Yacuambi Z1 (Cuadrilla. Nro. 8),False,2023-12-20 18:18:00,2023-12-20 18:30:00,0 days 00:12:00,False,False,"120,784",CABRERA GONZALEZ WILMER DAVID,R-142,"Zamora, San Francisco de Cumbaratza, Las Palmas",Orden de trabajo Yacuambi 20-12-2023 (WC).pdf
1147,511.05.001,En la agencia de la EERSSA Zamora se coordina ...,·,RUTINARIA,PROG,Zamora Z1 (Cuadrilla. AP Nro. 4),False,2023-12-11 08:00:00,2023-12-11 09:15:00,0 days 01:15:00,False,False,"120,147",MORALES RIVERA LUIS ALBERTO,R-151,Zamora,Orden de trabajo Alumbrado Zamora 11-12-2023 (...
3044,511.05.001,Se labora LP - RCL de 08:00 a 13:00 y de 14:00...,·,·,LABORA,Yantzaza (Agencia),False,2023-12-22 00:00:01,2023-12-22 00:00:02,0 days 00:00:01,False,False,"120,906",POMA GUAMAN LUIS ANTONIO,4-100,YANTZAZA - ZUMBI - LA FLORIDA.,Orden de trabajo Yantzaza 22-12-2023 (LP - Ele...
2773,511.05.001,"Agencia, recepción documentos, planificación a...",·,RUTINARIA,PROG,Gualaquiza (Agencia),False,2023-12-28 08:00:00,2023-12-28 08:30:00,0 days 00:30:00,False,False,"121,187",BUELE UYAGUARI CESAR CRISTIAN,R-12,"Gualaquiza La Paz, Guabi Alto",Orden de trabajo Gualaquiza 28-12-2023 (CB - E...
728,511.05.001,"Traslado del sector de Shiram Entnza, San Manu...",·,TRANSPORTE,TRANSP,Gualaquiza Z1 (Cuadrilla. Nro. 3),False,2023-12-03 16:05:00,2023-12-03 17:12:00,0 days 01:07:00,False,False,"119,774",LITUMA CORDOVA CESAR RUBEN,R-166,Gualaquiza.,Orden de trabajo Gualaquiza 03-12-2023 (CL).pdf
